# Technical demand response potential clustering

**Puporse and background**: This notebook serves for clustering the technical demand response potentials 
which were collected in a previous meta-analysis (Kochems 2020). This clustering routine is necessary since depicting all 
different units in the electricity market model would be computationally too expensive.

## Method applied
In the following, a brief description of the method applied is given for the sake of replicability.

### Filtering of demand response categories
The term "demand response categories" is introduced to describe the heterogeneous potential segmentation 
routines used in the publications evaluated within the meta-analysis. These demand response categories include 
processes, applications as well as entrire branches and mixtures of these categories.
* In the first step, data for entire branches is filtered out since in most cases there are too few data points and 
further information on availability is lacking. Thus, only data for processes and applications remain.
* In addition to that, categories covering different branches / appliances are filtered out since these would 
cause redundancies to publications analyzing the appliances in detail.

### Prepare and manipulate data for further usage (clustering / modeling)
In the second step, the data is manipulated for further usage in the clustering process as well as for the ensuing power market model analysis. This in turn consists of a few procedures:
* The data is combined to an overall data set and missing values are interpolated using the median value per sector. The parameters are filtered in order only to include those which are needed in demand response modeling in a power market model.
* A pairwhise correlation analysis using pearson's correlation coefficient is carried out in order to identify which parameters can be expressed through other ones since they show a high correlation. 
* Data is interpolated in order to remove inplausibilities. The data for the status quo is kept. The values for 2030 and 2050 are used to define trends in potential development. A linear interpolation is made in between. As far as the cost values are concerned, the values for the status quo as well as the minimum value are kept. The minimum value is assigned to 2050 and a linear interpolation is made in between.
> _Note: While the interpolation solution is preferable in terms of simplicity, it lacks some consistency: on the one hand in terms of a development of positive and negative potentials in line, on the other hand as for general trends within the sectors (such as declining industry production in the long run). This should be replaced by a more profound approach._

### Clustering of demand response categories
A clustering of demand response categories is carried out in the second step. 
A k-means clustering approach is used (as an alternative, it is possible to choose agglomerative clustering using ward linkage).<br>
Demand response categories are clustered using the (median values of the) following parameters (see also Steurer 2017, p. 83):
* shifting duration
* positive interference duration (shedding duration),
* variable costs,
* fixed costs and
* specific investments.

Some further aspects are worth mentioning:
* Negative interference duration is not taken into account because some processes are only eligible for load shedding and hence don't have a negative interference duration. In addition to that, a strong correlation between positive and negative interference duration has been detected.
* The clustering does not need to take into account the lower, middle and upper value for each parameter. A strong correlation between the values was determined which is why only the median values are used for the clustering. 
* Furthermore, the clustering is only carried out for the status quo and does not take development projections into account.
* The distinction between different sectors is kept. Some heating and cooling applications for tcs and households are combined since they comprise basically the same technology and creating identical clusters would not make much sense.
* For the aggregation of demand response parameters after clustering, a weighting by the available shifting resp. shedding capacity is carried out.

### Determination of availability
Since demand response potentials are time-dependent, availability has to be taken into account.<br>
For the analysis, the individual availability time series in hourly resolution of the original 
demand response categories are put together by calculating capacity weighted averages for the identified 
demand response clusters.

The availability time series are put together based on literature assumptions:
* The largest amount of the availability time series for individual demand response categories were created within three bachelor theses based on literature assumptions. They were put together in a separate notebook. The data output of this notebook in turn is read in here to form availability time series of demand response clusters.
* Some processes haven't been analyzed in the bachelor theses resp. the literature. For these, either existing availability time series of very similar demand response categories are assigned or own assumptions are made.
> _Note: These own assumptions for now are ("best guess") dummy values and should be replaced by more profound ones._
***

__*Important remaining TODOs:*__
* __*Revise parameters and potential development by including trend extrapolation / assumptions*__
* __*Revise proxies for availability time series*__
* __*Include the distinction between shifting and shedding &rarr; should be a criterion for separate cluster assignment*__

# Package imports
* Standard imports: scikit-learn (sklearn) is used for the clustering since it has built-in clustering routines, such as K-means
* User-defined functions:
    * *create_parameter_combinations*: Combines parameter names with measures of central tendency
    * *group_potential*: Does a grouping of the clusters determined using given aggregation functions per parameter
    * *wtrie_multiple_sheets*: Used to write multiple DataFrames as sheets at once into an Excel workbook
    * *map_column_names*: Maps column names of availability time series to the potential data column names.
    * *determine_missing_columns*: Lists the columns for which availability time series information is lacking and assumptions are needed.
    * *get_top_abs_correlations*: Determines the strongest correlation within a given correlation matrix.

In [2]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
from scipy.stats import linregress
from IPython.core.display import display

from potential_clustering_funcs import (
    create_parameter_combinations, group_potential, write_multiple_sheets,
    map_column_names, determine_missing_cols, assign_availability_remaining, get_top_abs_correlations
)

In [3]:
%matplotlib inline

# Parameter settings
Set path folder and filenames for reading in data.

Further parameters for controlling the workflow:
* *skip_dev*: If True, skip cells that are currently under development to keep overall functionality of notebook
* *write_outputs*: If True, outputs, i.e. demand response parameterizations resulting from the meta-analysis will be written into Excel workbooks
* *join_duplicates*: If True, demand response categories occuring in both, household and tcs sector will be combined together.
* *write_categories*: If True, the remaining categories will be written to Excel in order to match them with the
availability data.
* *adjust_potentials*: If True, availability time series information will be used to adjust the potential information. (I. e. if at max. 0.8 is reached, max potential will be set to 0.8 * max_potential)

Parameters for controlling the clustering routines:
* *cols*: Determine, which columns of the stats DataFrame shall be used for demand response parameterization
    * Usually lower and upper quartile as well as median are used.
> Note: This eliminates extreme values which are taken into account when min and max are used instead.<br>
The upper quartile of overall potential will be interpreted as maximum available potential. A possible alternative would be to use min and max values to depict extremes. &rarr; Advantage: would be more compelling. Drawback: Data lack for potential max.
* *cluster_parameters*: Determine, which demand response parameters to use for the clustering process. 
By default, these are the shifting duration, positive interference duration, variable costs as well as fixed costs.
* *cluster_algo*: The clustering algorithmn to be used ("KMeans" or "ward")
* *share_clusters*: Decide, how strong the original data will be reduced by giving a percentage of the original length. 
The cluster number is determined by the next higher integer. (Only applicable for k-means)
* *distance_threshold*: Decide, what distance threshold shall be set for the hierarchical clustering using ward 
linkage, i.e., when the algorithm should terminate.
* *print_clusters*: If True, prints out the clusters created (DataFrames)
* *use_ava_ts_for_profiles*: If True, availability time series in positive direction will be directly used to derive load profiles for the demand response categories resp. clusters, else profiles from the demand regio disaggregator will be applied

In [4]:
# Set path folder(s) and filename(s) for reading in / writing data
path_folder_in = "./"
path_folder_stats = "./out/stats/"
path_folder_availability = "./out/availability/"
path_folder_parameterization = "./out/parameterization/"
filename_in = "Potenziale_Lastmanagement.xlsx"
filename_availability_in = "availability_timeseries.xlsx"
filename_out = "parameterization"
filename_corr_out = "correlation"
filename_availability_out = "availability_timeseries_clusters.csv"
filename_load_profiles_out = "load_profile_timeseries_clusters"

# Set further parameters for controlling the workflow
skip_dev = True
write_outputs = True
join_duplicates = True
write_categories = True
adjust_potentials = True

# Determine clustering approach
cols = ["5%", "50%", "95%"]
cluster_parameters = [
    "shifting_duration", "interference_duration_pos", 
    "variable_costs", "fixed_costs", "specific_investments"
]

# Correlation analysis: optionally add potential_pos_overall (for all potential metrics)

cluster_algo = "KMeans"
share_clusters = 0.1
distance_threshold = 1000
print_clusters = True
use_ava_ts_for_profiles = True

# Read in and filter data
* Read in the categories data and filter out branches as well as power-to-X-technologies which won't be considered anymore.
* Read in the stats information on the demand response parameters from the previous meta-analysis.

> _NOTE: Data that is read in here could also be obtained directly from the DataFrames of the potential evaluation notebook. A separate notebook is used here only for the sake of shortness and readability._

## Read in and filter demand response categories
* Read in demand response categories
* Drop entire branches as well as categories conflicting with others or outside of scope (Power-to-X other than Power-to-Heat for space heating).
* Show the original number of categories and print the remaining number after filtering (without duplicates since categories may be used within different sectors).

In [5]:
categories = pd.read_excel(path_folder_in+filename_in, sheet_name="Kategorien_neu", index_col=0)

print("Number of original categories:\t{}".format(categories.shape[0]))

categories = categories[
    categories["Nutzung?"] == 1 
    & ~categories["Einstufung"].isin(["Branche", "Power-to-X"])
]

categories = categories.drop_duplicates(subset="Prozesskategorie")
categories = categories.set_index(["Prozesskategorie"], drop=True)

print("Number of remaining categories:\t{}".format(categories.shape[0]))

# Show the remaining demand response categories which are evaluated
list(categories.index.values)

Number of original categories:	99
Number of remaining categories:	38


['Chlor-Alkali-Elektrolyse',
 'Luftzerlegung',
 'Primäraluminiumelektrolyse',
 'Kupfer- und Zinkherstellung (Elektrolyse)',
 'Holz- und Zellstoffherstellung',
 'Altpapierrecycling (Pulper)',
 'Papiermaschinen',
 'Papierveredelung (Streichmaschinen und Kalander)',
 'Elektrostahlherstellung (Lichtbogenofen)',
 'Zementherstellung',
 'Kühlung (Lebensmittelindustrie)',
 'Belüftung',
 'Kühlhäuser',
 'Prozesskälte',
 'Klimakälte',
 'Warmwasserbereitstellung',
 'Nachtspeicherheizungen',
 'Notstromaggregate, Back-Up-Server und Mobilfunkstationen',
 'Waschmaschinen',
 'Wäschetrockner',
 'Geschirrspüler',
 'Kühlschränke',
 'Gefrierschränke und -truhen',
 'Wärmepumpen',
 'Heizungsumwälzpumpen',
 'Prozesswärme',
 'Druckluftanwendungen',
 'Abwasserbehandlung',
 'Kühl- und Gefrierkombinationen',
 'Calciumcarbid-Herstellung (Lichtbogenofen)',
 'Pumpenanwendungen in der Wasserversorgung',
 'Prozesskälte Handel',
 'Beleuchtung',
 'Pumpenanwendungen',
 'Gießereien (Induktionsofen)',
 'Zerkleinerer',
 'El

## Read in demand response parameters data
* Assign each demand response parameter an aggregation function to be used after clustering (sum or mean).
* Determine for which parameters to swap the order of preference.
> _Note: This is necessary, because in some cases minimum values are needed for an optimistic 
demand response projection and maximum for a pessimistic one, e.g. for minimum load factor. <br>
Hence, for these parameters, min is exchanged for max etc._
* Read in the data and store it in a dictionary indexed by demand response parameter. Hereby, a separation is necessary:
    * For the clustering process, certain appliances for the household and tcs sector should be combined, e.g. heat pumps. A new sector "tcs+hoho" is introduced for this purpose and the respective values are combined.
    * For calculating availabilities, a (re-)separation is necessary. This is because availability time series of the appliances may (slightly) differ among the sectors. &rarr; Herefore, a second dict is introduced.

In [6]:
# Assgin each parameter the aggregation function to be used
parameters_agg_dict = {
    "activation_duration": "mean", 
    "ave_load": "mean", 
    "fixed_costs": "mean", 
    "installed_cap": "sum",
    "interference_duration_neg": "mean", 
    "interference_duration_pos": "mean",
    "interference_duration_pos_shed": "mean",
    "max_load": "mean", 
    "maximum_activations_year": "mean", 
    "maximum_activations_year_shed": "mean",
    "min_load": "mean", 
    "potential_neg_overall": "sum",
    "potential_pos_overall": "sum",
    "potential_pos_overall_shed": "sum",
    "regeneration_duration": "mean", 
    "shiftable_share": "mean", 
    "shifting_duration": "mean",
    "specific_investments": "mean", 
    "variable_costs": "mean",
    "variable_costs_shed": "mean"
}

# Determine for each parameter whether or not to swap values
parameters_swap_dict = {
    "activation_duration": True, 
    "ave_load": True, 
    "fixed_costs": True, 
    "installed_cap": False,
    "interference_duration_neg": False, 
    "interference_duration_pos": False,
    "interference_duration_pos_shed": False,
    "max_load": False, 
    "maximum_activations_year": False, 
    "maximum_activations_year_shed": False,
    "min_load": True, 
    "potential_neg_overall": True,
    "potential_pos_overall": False,
    "potential_pos_overall_shed": False,
    "regeneration_duration": True, 
    "shiftable_share": False, 
    "shifting_duration": False,
    "specific_investments": True, 
    "variable_costs": True,
    "variable_costs_shed": True  
}

# Map columns for swapping
swap_cols = {
    "min": "max",
    "5%": "95%",
    "10%": "90%",
    "25%": "75%"
}

sectors = ["ind", "tcs", "hoho"]

years = ["SQ", "2020", "2025", "2030", "2035", "2040", "2045", "2050"]

to_join = ["Nachtspeicherheizungen", "Warmwasserbereitstellung", "Wärmepumpen", "Klimakälte"]
to_drop = {"Prozesskälte": "hoho"}

> _NOTE: While the fundamental routine here is quite okay and fast enough, there is room for improvement in the aggregation routine for the duplicates DataFrame, i.e. the DataFrame containing data for tcs+hoho which is then aggregated. Fo a compelling approach, a capacity weighted average should be calaculated using overall positive potential as capacity information. In order to do so, capacity information has to be added to the individual data sets._

In [7]:
categories.index

Index(['Chlor-Alkali-Elektrolyse', 'Luftzerlegung',
       'Primäraluminiumelektrolyse',
       'Kupfer- und Zinkherstellung (Elektrolyse)',
       'Holz- und Zellstoffherstellung', 'Altpapierrecycling (Pulper)',
       'Papiermaschinen', 'Papierveredelung (Streichmaschinen und Kalander)',
       'Elektrostahlherstellung (Lichtbogenofen)', 'Zementherstellung',
       'Kühlung (Lebensmittelindustrie)', 'Belüftung', 'Kühlhäuser',
       'Prozesskälte', 'Klimakälte', 'Warmwasserbereitstellung',
       'Nachtspeicherheizungen',
       'Notstromaggregate, Back-Up-Server und Mobilfunkstationen',
       'Waschmaschinen', 'Wäschetrockner', 'Geschirrspüler', 'Kühlschränke',
       'Gefrierschränke und -truhen', 'Wärmepumpen', 'Heizungsumwälzpumpen',
       'Prozesswärme', 'Druckluftanwendungen', 'Abwasserbehandlung',
       'Kühl- und Gefrierkombinationen',
       'Calciumcarbid-Herstellung (Lichtbogenofen)',
       'Pumpenanwendungen in der Wasserversorgung', 'Prozesskälte Handel',
       'Bel

## UPDATE: Update / facilitate the processing routine

Create a common data basis

In [8]:
# Put everything into one common pandas.DataFrame
all_params_df = pd.DataFrame()

# Count the number of params for which no data is available
count_ignored = 0

for parameter, swap_param in parameters_swap_dict.items():
    for year in years:
        for sector in sectors:
            try:
                new_df = pd.read_csv(
                    path_folder_stats+parameter+"_"+sector+"_stats_" + year + ".csv",
                    sep=";",
                    index_col=0
                ).T
                new_df.index.name = "Prozesskategorie"

                # Change the order of appearance if swap is needed for parameter
                if swap_param:
                    for k, v in swap_cols.items():
                        new_df[k + "_copy"] = new_df[k]
                        new_df[v + "_copy"] = new_df[v]
                        new_df[k] = new_df[v + "_copy"]
                        new_df[v] = new_df[k + "_copy"]
                        new_df.drop(columns=[k + "_copy", v + "_copy"], inplace=True)
                    
                new_df["parameter"] = parameter
                new_df["sector"] = sector
                new_df["year"] = year
                all_params_df = pd.concat([all_params_df, new_df], sort="False")
            except:
                count_ignored += 1
                continue

print("Overall number of params (sectors, years, params): " + str(len(parameters_agg_dict) * (len(sectors)) * len(years)))
print("Number of params not eligible for evaluation: " + str(count_ignored))

Overall number of params (sectors, years, params): 480
Number of params not eligible for evaluation: 114


Drop values

In [9]:
# Filter out the categories to be used and drop certain ones for dedicated sector only
all_params_df = all_params_df.loc[all_params_df.index.isin(categories.index)]
all_params_df.set_index("sector", append=True, inplace=True)
all_params_df.drop(index={(k, v) for k, v in to_drop.items()}, inplace=True)
all_params_df.reset_index(level=1, inplace=True)

Combine duplicates for tcs & hoho

In [10]:
# Extend index and filter for duplicate values
all_params_df.set_index(["year", "parameter"], append=True, inplace=True)
filter_duplicates = (
    all_params_df.index.duplicated(keep=False)
    & all_params_df.sector.isin(["hoho", "tcs"])
)

# Get duplicates within duplicates (i.e. filter out the values where 
# the same demand response categories occur in tcs and industry sector)
duplicates_df = all_params_df.loc[filter_duplicates]
duplicates_df = duplicates_df[duplicates_df.index.duplicated(keep=False)]

# Filter out the remaining duplicate values from original DataFrame
keys = list(duplicates_df.columns.values)
i1 = all_params_df.set_index(keys).index
i2 = duplicates_df.set_index(keys).index
no_duplicates_df = all_params_df.loc[~i1.isin(i2)]

# Assign certain demand response categories to a combined tcs & hoho sector ("tcs+hoho")
for el in to_join:
    if el in no_duplicates_df.index:
        no_duplicates_df.loc[:,"help_sector"] = np.where(
            no_duplicates_df.loc[:,"sector"].values == "ind", 
            "ind", 
            "tcs+hoho"
        )
        no_duplicates_df.loc[el, "sector"] = no_duplicates_df.loc[el, "help_sector"].values
        no_duplicates_df = no_duplicates_df.drop(["help_sector"], axis=1)

C:\Users\koch_j0\.conda\envs\diss3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\koch_j0\.conda\envs\diss3\lib\site-packages\pandas\core\indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
<ipython-input-10-c942b6c9a8f0>:27: PerformanceWarning: indexing past lexsort depth may impact performance.
  no_duplicates_df.loc[el, "sector"] = no_duplicates_df.loc[el, "help_sector"].v

In [11]:
# Drop index for grouping
duplicates_df.reset_index(drop=False, inplace=True)

# Introduce DataFrame for grouping duplicates (sector "tcs+hoho")
grouped_duplicates_df = pd.DataFrame()

for parameter, param_agg_rule in parameters_agg_dict.items():
    param_duplicates_df = duplicates_df.loc[duplicates_df["parameter"] == parameter]
    if not param_duplicates_df.empty:
        param_duplicates_df = param_duplicates_df.groupby(["Prozesskategorie", "year", "parameter"]).agg({
            "count": "sum",
            "min": param_agg_rule,
            "5%": param_agg_rule,
            "10%": param_agg_rule,
            "25%": param_agg_rule,
            "50%": param_agg_rule,
            "mean": param_agg_rule,
            "std": param_agg_rule,
            "75%": param_agg_rule,
            "90%": param_agg_rule,
            "95%": param_agg_rule,
            "max": param_agg_rule,
        })
        grouped_duplicates_df = pd.concat([grouped_duplicates_df, param_duplicates_df])
        
grouped_duplicates_df["sector"] = "tcs+hoho"

# Combine the non-duplicated data with the consolidated tcs+hoho data
all_params_df = pd.concat([no_duplicates_df, grouped_duplicates_df], sort=False)

# Create unique index and round to four digits
all_params_df.set_index("sector", append=True, inplace=True)
all_params_df = all_params_df.round(4)

sectors.append("tcs+hoho")

In [12]:
all_params_df

10%  \
Prozesskategorie                           year parameter            sector               
Altpapierrecycling (Pulper)                SQ   activation_duration  ind         1.8858   
Calciumcarbid-Herstellung (Lichtbogenofen) SQ   activation_duration  ind         0.2500   
Chlor-Alkali-Elektrolyse                   SQ   activation_duration  ind         0.6250   
Elektrostahlherstellung (Lichtbogenofen)   SQ   activation_duration  ind         1.6000   
Gießereien (Induktionsofen)                SQ   activation_duration  ind         0.2500   
...                                                                                 ...   
Warmwasserbereitstellung                   2030 shifting_duration    tcs+hoho    4.9100   
                                           2050 shifting_duration    tcs+hoho    4.4400   
                                           SQ   shifting_duration    tcs+hoho    2.0875   
Klimakälte                                 2020 specific_investments tcs+hoho  132.3576   
                                           SQ   specific_investments tcs+hoho  606.7025   

                                                                                    25%  \
Prozesskategorie                           year parameter            sector               
Altpapierrecycling (Pulper)                SQ   activation_duration  ind         0.7038   
Calciumcarbid-Herstellung (Lichtbogenofen) SQ   activation_duration  ind         0.2292   
Chlor-Alkali-Elektrolyse                   SQ   activation_duration  ind         0.2500   
Elektrostahlherstellung (Lichtbogenofen)   SQ   activation_duration  ind         0.8220   
Gießereien (Induktionsofen)                SQ   activation_duration  ind         0.2292   
...                                                                                 ...   
Warmwasserbereitstellung                   2030 shifting_duration    tcs+hoho    7.0000   
                                           2050 shifting_duration    tcs+hoho    6.7500   
                                           SQ   shifting_duration    tcs+hoho    5.0000   
Klimakälte                                 2020 specific_investments tcs+hoho  105.3630   
                                           SQ   specific_investments tcs+hoho  422.1312   

                                                                                     5%  \
Prozesskategorie                           year parameter            sector               
Altpapierrecycling (Pulper)                SQ   activation_duration  ind         2.4429   
Calciumcarbid-Herstellung (Lichtbogenofen) SQ   activation_duration  ind         0.2500   
Chlor-Alkali-Elektrolyse                   SQ   activation_duration  ind         1.0000   
Elektrostahlherstellung (Lichtbogenofen)   SQ   activation_duration  ind         2.2667   
Gießereien (Induktionsofen)                SQ   activation_duration  ind         0.2500   
...                                                                                 ...   
Warmwasserbereitstellung                   2030 shifting_duration    tcs+hoho    4.3200   
                                           2050 shifting_duration    tcs+hoho    3.6000   
                                           SQ   shifting_duration    tcs+hoho    0.9812   
Klimakälte                                 2020 specific_investments tcs+hoho  141.3559   
                                           SQ   specific_investments tcs+hoho  668.2262   

                                                                                   50%  \
Prozesskategorie                           year parameter            sector              
Altpapierrecycling (Pulper)                SQ   activation_duration  ind        0.3746   
Calciumcarbid-Herstellung (Lichtbogenofen) SQ   activation_duration  ind        0.1667   
Chlor-Alkali-Elektrolyse                   SQ   activation_duration  ind        0.2221   
Elektrostahlherstellung (Lichtbogenofen)   SQ   activation_duration  ind        0.

# Prepare and manipulate data for further usage

## UPDATE: Facilitate joining data
> _**NOTE**:_ 
> * In the previous version, there were obviously some problems with combining the data that led to nan entries which had to be filled in order to be able to cluster.
> * In the current version, there is already a combined data set created from the routine above. Obtaining status quo data is just a simple single line filtering task, so it seems that the following section can be entirely skipped and is not needed anymore.

In [15]:
all_params_status_quo = all_params_df.loc[all_params_df.index.get_level_values(1) == "SQ"]

In [27]:
test = all_params_status_quo.reset_index()

In [28]:
test

,Prozesskategorie,year,parameter,sector,10%,25%,5%,50%,75%,90%,95%,count,max,mean,min,std
0,Altpapierrecycling (Pulper),SQ,activation_duration,ind,1.8858,0.7038,2.4429,0.3746,0.1248,0.0458,0.0271,6.0,0.0083,0.7687,3.000,1.1286
1,Calciumcarbid-Herstellung (Lichtbogenofen),SQ,activation_duration,ind,0.2500,0.2292,0.2500,0.1667,0.1042,0.0833,0.0833,6.0,0.0833,0.1667,0.250,0.0745
2,Chlor-Alkali-Elektrolyse,SQ,activation_duration,ind,0.6250,0.2500,1.0000,0.2221,0.0552,0.0001,0.0000,16.0,0.0000,0.2512,1.000,0.3102
3,Elektrostahlherstellung (Lichtbogenofen),SQ,activation_duration,ind,1.6000,0.8220,2.2667,0.1667,0.0002,0.0001,0.0001,12.0,0.0000,0.5845,3.000,0.9240
4,Gießereien (Induktionsofen),SQ,activation_duration,ind,0.2500,0.2292,0.2500,0.1667,0.1042,0.0833,0.0833,6.0,0.0833,0.1667,0.250,0.0745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,Warmwasserbereitstellung,SQ,potential_pos_overall,tcs+hoho,291.2460,683.7500,265.5460,3580.0000,5336.4800,6259.3400,6268.8400,45.0,12865.0000,3437.6950,239.846,3197.2188
425,Klimakälte,SQ,shifting_duration,tcs+hoho,0.3025,0.3625,0.2762,0.3750,0.7625,1.3100,1.6550,13.0,2.0000,0.7031,0.250,0.6616
426,Nachtspeicherheizungen,SQ,shifting_duration,tcs+hoho,2.0625,4.2250,1.3750,7.5500,17.5000,19.0000,19.5000,22.0,20.0000,9.7328,0.250,7.5023
427,Warmwasserbereitstellung,SQ,shifting_duration,tcs+hoho,2.0875,5.0000,0.9812,9.0000,17.5000,19.0000,19.5000,23.0,20.0000,10.7390,0.625,7.5808


In [29]:
test.pivot(index="Prozesskategorie", columns=["parameter"])

ValueError: Index contains duplicate entries, cannot reshape

In [19]:
all_params_status_quo.index.get_level_values(2).unique()

20

## Original routing (KEEP FOR NOW; REMOVE WHEN UPDATE IS WORKING)
See following section

## Put together parameter info and fill missing values
* Put together all parameter information for a given sector in order to prepare for clustering
* Deal with missing values:
    * Calculate averages per sector and assing these.
    * If information is still lacking for the status quo, fill remaining values with 0s for the status quo year. Since this is only applicable to the parameters activation duration, regeneration duration and variable costs, this causes no problems. Variable costs values of 0 are replaced later.
* Round data to two decimal places.

> _Note: In general, there are several options possible for dealing with missing data:_
> * Drop missing values &rarr; Is not recommended and would eliminate all data
> * Fill NaN values (and round data)
>    * Use 0 values to fill NaN &rarr; changes the cluster decision itself 
>    * Use averages per sector &rarr; changes the cluster decision itself (approach is used at the moment)
>    * Use imputations: https://scikit-learn.org/stable/modules/impute.html &rarr; state of the art but easy to make mistakes
>    * Alternative to filling missing values: Partial cluster analysis (not supported by scikit-learn).
>
> See for options to deal with missing data: https://www.displayr.com/5-ways-deal-missing-data-cluster-analysis/

In [ ]:
# Create a separate DataFrame for every year key and store it in potential dict (indexed by year keys)
# The clustering is carried out based on status quo data
potential_dict = {}
potential_dict_nojoin = {}

for year in years:
    # Create a DataFrame to store the information on all demand response parameters
    df_potential = pd.DataFrame(columns=["Prozess", "sector", "year_key"]).set_index("Prozess")
    df_potential_nojoin = pd.DataFrame(columns=["Prozess", "sector", "year_key"]).set_index("Prozess")

    cols_new = cols.copy()
    cols_new.extend(["sector", "year_key"])

    # Put together the information for all demand response parameters
    for k, v in parameters_dict.items():
        try:
            to_merge = v[cols_new][v["year_key"] == year]
            if not to_merge.empty:
                df_potential = pd.merge(df_potential, to_merge, on=["Prozess", "sector", "year_key"], 
                                        suffixes=["","_"+k], how="outer")
        except:
            continue
            
    # Put together the information for all demand response parameters (not joined version)
    for k, v in parameters_dict_nojoin.items():
        try:
            to_merge = v[cols_new][v["year_key"] == year]
            if not to_merge.empty:
                df_potential_nojoin = pd.merge(df_potential_nojoin, to_merge, on=["Prozess", "sector", "year_key"], 
                                               suffixes=["","_"+k], how="outer")
        except:
            continue

    # Rename data for the first parameter (activation duration) which is lost in the merging process
    # Set process and sector as index to obtain unique values
    df_potential = df_potential.rename(columns={col: col+"_"+list(parameters_groups_dict.keys())[0] 
                                                for col in cols})
    
    df_potential_nojoin = df_potential_nojoin.rename(columns={col: col+"_"+list(parameters_groups_dict.keys())[0] 
                                                          for col in cols})
    
    potential_dict[year] = df_potential.set_index([df_potential.index, "sector"])
    potential_dict_nojoin[year] = df_potential_nojoin.set_index([df_potential_nojoin.index, "sector"])
    
    # Fill Nan values and round:
    # The mean values per sector and year are assigned to nans. 
    # If data is lacking completely, 0s are used to fill up the gaps (only for the status quo)
    for sector in sectors:
        median = potential_dict[year][potential_dict[year].index.get_level_values(1) == sector].median()
        potential_dict[year][potential_dict[year].index.get_level_values(1) == sector] = \
            potential_dict[year][potential_dict[year].index.get_level_values(1) == sector].fillna(median)
        if year == "SQ":
            potential_dict[year] = potential_dict[year].fillna(0)
        
        median_nj = potential_dict_nojoin[year][potential_dict_nojoin[year].index.get_level_values(1) == sector].median()
        potential_dict_nojoin[year][potential_dict_nojoin[year].index.get_level_values(1) == sector] = \
            potential_dict_nojoin[year][potential_dict_nojoin[year].index.get_level_values(1) == sector].fillna(median_nj)
        if year == "SQ":
            potential_dict_nojoin[year] = potential_dict_nojoin[year].fillna(0)

    potential_dict[year] = potential_dict[year].round(2)
    potential_dict_nojoin[year] = potential_dict_nojoin[year].round(2)

In [ ]:
potential_dict["SQ"].head()

## Do a correlation analysis of parameters in order to derive cluster parameters
General idea:
* Determine  pairwhise correlation in order to determine which parameters to use for clustering.
* If correlation between to parameters is above 0.8, choose one of both.

## EXPERIMENTAL

In [16]:
corr_matrix = all_params_status_quo.corr().round(2)
display(corr_matrix)

,10%,25%,5%,50%,75%,90%,95%,count,max,mean,min,std
10%,1.00,0.98,0.99,0.96,0.93,0.91,0.90,0.03,0.88,0.96,0.87,0.78
25%,0.98,1.00,0.96,0.99,0.97,0.94,0.93,-0.01,0.92,0.99,0.82,0.76
5%,0.99,0.96,1.00,0.94,0.91,0.88,0.87,0.06,0.86,0.95,0.89,0.78
50%,0.96,0.99,0.94,1.00,0.99,0.95,0.94,-0.03,0.93,0.99,0.78,0.75
75%,0.93,0.97,0.91,0.99,1.00,0.99,0.98,-0.02,0.97,0.99,0.75,0.82
90%,0.91,0.94,0.88,0.95,0.99,1.00,1.00,-0.02,1.00,0.98,0.73,0.88
95%,0.90,0.93,0.87,0.94,0.98,1.00,1.00,-0.01,1.00,0.98,0.72,0.89
count,0.03,-0.01,0.06,-0.03,-0.02,-0.02,-0.01,1.00,0.01,-0.00,0.21,0.13
max,0.88,0.92,0.86,0.93,0.97,1.00,1.00,0.01,1.00,0.97,0.71,0.90
mean,0.96,0.99,0.95,0.99,0.99,0.98,0.98,-0.00,0.97,1.00,0.80,0.84


## Original routine (KEEP FOR NOW; REMOVE WHEN UPDATE IS WORKING)

In [ ]:
# Create and display correlation matrix
corr_matrix = potential_dict["SQ"].corr().round(2)
display(corr_matrix)

In [ ]:
# Display the top 10 largest correlations
print("Top Absolute Correlations")
print(75 * "-")
display(get_top_abs_correlations(corr_matrix, n=10))

In [ ]:
top_corr_series = get_top_abs_correlations(corr_matrix, threshold=0.8)

In [ ]:
corr_dict = {}
ix1_slice_end = top_corr_series.index.get_level_values(0).str.slice(start=4)
ix2_slice_end = top_corr_series.index.get_level_values(1).str.slice(start=4)
ix1_slice_start = top_corr_series.index.get_level_values(0).str.slice(stop=4)
ix2_slice_start = top_corr_series.index.get_level_values(1).str.slice(stop=4) 

corr_dict["same_params"] = top_corr_series[ix1_slice_end == ix2_slice_end]
corr_dict["different_params"] = top_corr_series[ix1_slice_end != ix2_slice_end]
corr_dict["only_medians"] = top_corr_series[(ix1_slice_end != ix2_slice_end)
                                            & (ix1_slice_start == "50%_") & (ix2_slice_start == "50%_")]

if write_outputs:
    write_multiple_sheets(corr_dict, path_folder_parameterization, filename_corr_out + ".xlsx")

In [ ]:
# Show heatmap for correlations
fig, ax = plt.subplots(figsize=(20, 15))
_ = sns.heatmap(corr_matrix, cmap="RdGy", vmin=-1.0, vmax=1.0)
plt.show()

## Create overall data set and interpolate missing values prior to clustering
Overview on approach used:
1. Create an **overall data set** and slice the parameters of interest.
2. Introduce **linear interpolation** for potential and cost parameters:
    * Potential parameters: The values for the status quo, 2030 and 2050 are used. Linear interpolation is made in between.
    * Cost parameters: The minimum cost value is assigned to 2050 since cost increases seem pretty unlikely. Linear interpolation is made between the status quo and 2050 (minimum cost value). Costs of zero are set to 0.01 in order to attribute cost to DR and prevent numeric effects disturbing DR utilization in the model.
3. **Assign remaining** parameters (mostly time-related ones) the same as for the status quo for every year.


> _Note: There is quite some room for **further improvements**._
> * _Development of positive and negative potentials should be in line with each other which doesn't have to be the case in the approach used._
> * _General trends in parameter development could be identified (increase / decrease / constant) by some other systematic like linear regression (some code snippets are kept, but not used for now here)._

### Create overall data set

In [ ]:
# Introduce a DataFrame to store overall data for all years
potential_all_years = pd.DataFrame()
potential_all_years_nojoin = pd.DataFrame()

for year in years:
    potential_dict[year]["year_key"] = year
    potential_dict_nojoin[year]["year_key"] = year
    potential_all_years = pd.concat([potential_all_years, potential_dict[year]])
    potential_all_years_nojoin = pd.concat([potential_all_years_nojoin, potential_dict_nojoin[year]])

In [ ]:
# Get all demand response categories
dr_cats = potential_all_years.index.unique().values

Only a **subset of the parameters** needs to be further analyzed:
* Time-dependent parameters are assumed constant. These comprise:
    * activation duration
    * interference duration (both pos and neg) and shifting duration
    * regeneration duration
* Other parameters are not really resp. not directly used in the modeling approaches for DR. These comprise:
    * average, minimum and maximum load
* This leads to the following remaining parameters focussing on costs and potentials. Since the correlation analysis showed high redundandencies for the potential parameters, only the following remaining parameters will be further analyzed:
    * potential positive overall
    * potential negative overall
    * installed capacity
    * fixed and variable costs
    * specific investments

In [ ]:
dr_cats_nojoin = potential_all_years_nojoin.index.unique().values

### Linear interpolation for potential and costs data

In [ ]:
# choose parameters for which some adaptions are needed
params_tbu_list = ["potential_neg_overall",
                   "potential_pos_overall",
                   "installed_cap",
                   "fixed_costs", "variable_costs", "specific_investments"]

params_tbu = create_parameter_combinations(params_tbu_list, cols)
params_tbu.append("year_key")

In [ ]:
# Slice the parameter values needed
slice_pot = potential_all_years.loc[:,params_tbu]
slice_pot_nojoin = potential_all_years_nojoin.loc[:,params_tbu]
neg_pot_cols = create_parameter_combinations(["potential_neg_overall"], cols)
slice_pot[neg_pot_cols] = slice_pot[neg_pot_cols].mul(-1)
slice_pot_nojoin[neg_pot_cols] = slice_pot_nojoin[neg_pot_cols].mul(-1)

Perform **interpolation on potential and costs parameters** in order to
1. Fill data gaps and
2. remove inplausibilities such as potentials changing very strong and not consistent within the five year intervalls used.

In [ ]:
# Define potential and cost cols
pot_cols = create_parameter_combinations(["potential_pos_overall", 
                                          "potential_neg_overall", 
                                          "installed_cap"], cols)
cost_cols = create_parameter_combinations(["specific_investments", 
                                           "fixed_costs", 
                                           "variable_costs"], cols)

# Instanciate new DataFrame to store manipulated outputs
slice_pot_new = pd.DataFrame(index=pd.MultiIndex.from_product([[],[],[]], 
                             names=["Prozess", "sector", "year_key"]))
slice_pot_new_nojoin = pd.DataFrame(index=pd.MultiIndex.from_product([[],[],[]], 
                                    names=["Prozess", "sector", "year_key"]))

# Create a list of DataFrames to concat
to_concat = [slice_pot_new]
to_concat_nojoin = [slice_pot_new_nojoin]

for el in dr_cats:
    df = slice_pot.loc[el].set_index("year_key", append=True)

    # Check potential values for interpolation
    try:
        pot_vals_SQ = df.loc[(el[0], el[1], "SQ"), pot_cols]
    except KeyError:
        continue
    try:
        pot_vals_2030 = df.loc[(el[0], el[1], "2030"), pot_cols]
    except:
        pot_vals_2030 = np.nan
    try:
        pot_vals_2050 = df.loc[(el[0], el[1], "2050"), pot_cols]
    except:
        pot_vals_2050 = pot_vals_SQ
    
    # Assign potential values for 2030 and 2050 and interpolate in between
    multi_ix = pd.MultiIndex.from_product([[el[0]], [el[1]], years], 
                                          names=["Prozess", "sector", "year_key"])
    new_df = pd.DataFrame(index=multi_ix)
    new_df.loc[(el[0], el[1], "SQ"), pot_cols] = pot_vals_SQ
    new_df.loc[(el[0], el[1], "2030"), pot_cols] = pot_vals_2030
    new_df.loc[(el[0], el[1], "2050"), pot_cols] = pot_vals_2050

    # Check cost values for interpolation
    cost_vals_SQ = df.loc[(el[0], el[1], "SQ"), cost_cols]     
    min_costs = df[cost_cols].min()

    # Assign minimum value for 2050 and interpolate in between
    new_df.loc[(el[0], el[1], "SQ"), cost_cols] = cost_vals_SQ.fillna(0.01)
    new_df.loc[(el[0], el[1], "2050"), cost_cols] = min_costs
    new_df[cost_cols] = new_df[cost_cols].replace({0: 0.01})
    new_df = new_df.interpolate()

    new_df = new_df.interpolate()
    to_concat.append(new_df)

slice_pot_new = pd.concat([el for el in to_concat], levels=([0, 1, 2]))    

# nojoin
for el in dr_cats_nojoin:
    df = slice_pot_nojoin.loc[el].set_index("year_key", append=True)

    # Check potential values for interpolation
    try:
        pot_vals_SQ = df.loc[(el[0], el[1], "SQ"), pot_cols]
    except KeyError:
        continue
    try:
        pot_vals_2030 = df.loc[(el[0], el[1], "2030"), pot_cols]
    except:
        pot_vals_2030 = np.nan
    try:
        pot_vals_2050 = df.loc[(el[0], el[1], "2050"), pot_cols]
    except:
        pot_vals_2050 = pot_vals_SQ
    
    # Assign potential values for 2030 and 2050 and interpolate in between
    multi_ix = pd.MultiIndex.from_product([[el[0]], [el[1]], years], 
                                          names=["Prozess", "sector", "year_key"])
    new_df = pd.DataFrame(index=multi_ix)
    new_df.loc[(el[0], el[1], "SQ"), pot_cols] = pot_vals_SQ
    new_df.loc[(el[0], el[1], "2030"), pot_cols] = pot_vals_2030
    new_df.loc[(el[0], el[1], "2050"), pot_cols] = pot_vals_2050

    # Check cost values for interpolation
    cost_vals_SQ = df.loc[(el[0], el[1], "SQ"), cost_cols]     
    min_costs = df[cost_cols].min()

    # Assign minimum value for 2050 and interpolate in between
    new_df.loc[(el[0], el[1], "SQ"), cost_cols] = cost_vals_SQ.fillna(0.01)
    new_df.loc[(el[0], el[1], "2050"), cost_cols] = min_costs
    new_df[cost_cols] = new_df[cost_cols].replace({0: 0.01})
    new_df = new_df.interpolate().round(2)
    
    new_df = new_df.interpolate()
    to_concat_nojoin.append(new_df)
    
slice_pot_new_nojoin = pd.concat([el for el in to_concat_nojoin], levels=([0, 1, 2]))  

In [ ]:
slice_pot_new[cost_cols].min()

**Alternative trend analysis for DR parameters**:
> _NOTE: This section is work in progress and contains some open questions._<br>
> _It will be skipped for that reason (unless it is explicitly stated not to be skipped)._
* Potential parameters:
    * If $ r^2 \geq 0.8 $ &rarr; apply linear regression.
    * Use other approach yet to be defined...
* Cost parameters: Suitable approach yet to be defined...

In [ ]:
# Alternative: use linear regression to analyze trends for dr categories
if not skip_dev:
    # "year_key" has to be an integer for that purpose
    slice_pot["year_key"] = slice_pot["year_key"].replace({"SQ": "2015"})
    slice_pot["year_key"] = slice_pot["year_key"].astype(int)
    
    for el in dr_cats:
        for pot_col in ["50%_potential_neg_overall", "50%_potential_pos_overall"]:
            slope, intercept, r_value, p_value, std_err = linregress(
                slice_pot.loc[el, "year_key"].values,
                slice_pot.loc[el, pot_col].values)
            if r_value ** 2 >= 0.8:
                slice_pot.loc[el, pot_col][slice_pot.loc[el, "year_key"] != 2015] \
                    = intercept + slope * slice_pot.loc[el, "year_key"]

        #plt.scatter(slice_pot.loc[el, "year_key"].values, 
        #            slice_pot.loc[el, pot_col].values)

### Assignment of remaining (time-related parameters)

In [ ]:
# Assign remaining (mostly time-related) parameters
params_remaining_list = ["interference_duration_neg",
                         "interference_duration_pos", 
                         "maximum_activations_year", 
                         "regeneration_duration", 
                         "shifting_duration"]

params_remaining = create_parameter_combinations(params_remaining_list, cols)
params_remaining.append("year_key")

In [ ]:
# Slice the parameter values needed
slice_pot = potential_all_years.loc[:,params_remaining]
slice_pot_nojoin = potential_all_years_nojoin.loc[:,params_remaining]
slice_pot.set_index("year_key", append=True, drop=False, inplace=True)
slice_pot_nojoin.set_index("year_key", append=True, drop=False, inplace=True)
params_remaining.remove("year_key")

In [ ]:
slice_pot_new.loc[slice_pot_new.index.get_level_values(2) == "SQ", params_remaining] =\
    slice_pot.loc[slice_pot.index.get_level_values(2) == "SQ", params_remaining]
slice_pot_new.fillna(method="ffill", inplace=True)

slice_pot_new_nojoin.loc[slice_pot_new_nojoin.index.get_level_values(2) == "SQ", params_remaining] =\
    slice_pot_nojoin.loc[slice_pot_nojoin.index.get_level_values(2) == "SQ", params_remaining]
slice_pot_new_nojoin.fillna(method="ffill", inplace=True)

### Rearrange to dict-based data structure
> _NOTE: In general, this is not a necessity, but utilized here in order not to have to revert the entire ensuing clustering routine (which was initially build up upon the dict-based data structure)._

In [ ]:
slice_pot_new = slice_pot_new.reset_index(level=2)
slice_pot_new_nojoin = slice_pot_new_nojoin.reset_index(level=2)

for year in years:
    potential_dict[year] = slice_pot_new[slice_pot_new["year_key"] == year].drop("year_key", axis=1)
    potential_dict_nojoin[year] = slice_pot_new_nojoin[slice_pot_new_nojoin["year_key"] == year].drop("year_key", axis=1)

In [ ]:
potential_dict["SQ"].head()

In [ ]:
potential_dict_nojoin["2050"].tail()

In [ ]:
potential_dict["SQ"]

# Cluster data
* Do a clustering using K-Means and the cluster parameters defined in the above parameter settings (alternative: agglomerative clustering using ward linkage).
* Default: Cluster within sectors by shifting times, positive interference duration, variable costs as well as specific investments

## Do the actual clustering
* Create a list of the cluster parameters
* Determine the number of clusters per sector
* Perform a K-Means clustering on the data (alternative: agglomerative clustering using ward linkage)

In [ ]:
cluster_parameters_list = create_parameter_combinations(cluster_parameters, ["50%"])
cluster_parameters_list

In [ ]:
# Initialize the column to store the cluster label
potential_dict["SQ"]["cluster"] = 0

print(f"Number of clusters for {cluster_algo} clustering")

# Increase cluster numbers by (arbitratily chosen) increments
# in order not to overwrite cluster information
iter = 0
increments = [1000, 100, 10, 1]

# Iterate over the sectors and create clusters within the sectors
for sector in potential_dict["SQ"].index.get_level_values(1).unique():
    # Determine the number of clusters per sector and print it
    n = math.ceil(potential_dict["SQ"][potential_dict["SQ"].index.get_level_values(1) == sector].shape[0]*share_clusters)
    print(sector+":\t\t"+str(n))
    
    if cluster_algo == "KMeans":
        # Do the actual clustering and assign the cluster labels
        #display(potential_dict["SQ"].loc[potential_dict["SQ"].index.get_level_values(1) == sector])
        potential_dict["SQ"].loc[potential_dict["SQ"].index.get_level_values(1) == sector, "cluster"] = \
            KMeans(n_clusters=n).fit(
                potential_dict["SQ"].loc[potential_dict["SQ"].index.get_level_values(1) == sector, 
                                         cluster_parameters_list].values).labels_ + 1
        
    elif cluster_algo == "ward":
        # Do the actual clustering and assign the cluster labels
        potential_dict["SQ"].loc[potential_dict["SQ"].index.get_level_values(1) == sector, "cluster"] = \
            AgglomerativeClustering(n_clusters=n, linkage="ward").fit(
                potential_dict["SQ"].loc[potential_dict["SQ"].index.get_level_values(1) == sector, cluster_parameters_list].values).labels_ + 1       

    # Increment the cluster labels to avoid overwriting in the next iteration (K-Means always starts with zero)
    potential_dict["SQ"].loc[potential_dict["SQ"]["cluster"] != 0, "cluster"] += n * increments[iter]
    iter += 1
    #display(potential_dict["SQ"].loc[potential_dict["SQ"].index.get_level_values(1) == sector])

In [ ]:
# Print the number of unique cluster labels (cross check)
potential_dict["SQ"]["cluster"].nunique()

**Cluster assignment**:
* Assign cluster information for future years (_Note: simple assignment does an index mapping_).
* Furthermore, assing cluster information to data set in which tcs and hoho applications haven't been joined.

In [ ]:
for year in years:
    if not year == "SQ":
        potential_dict[year]["cluster"] = potential_dict["SQ"]["cluster"]
        
    # Add cluster information for data not joined
    potential_dict_nojoin[year]["cluster"] = potential_dict[year]["cluster"]

    # Add cluster information for categories for which it is missing, i.e. categories in tcs+hoho sector
    # Determine indices to use for selecting cluster numbers
    no_cluster_ixs = list(set(potential_dict_nojoin[year][
        potential_dict_nojoin[year]["cluster"].isna()].index.get_level_values(0)))
    no_cluster_ixs = [(i, j) for i in no_cluster_ixs for j in ["tcs+hoho"]]

    # Use only the first index level to be able to assign values (second index levels won't match)
    clusters_to_use = potential_dict[year].loc[no_cluster_ixs, "cluster"].to_frame()
    clusters_to_use = clusters_to_use.set_index(clusters_to_use.index.get_level_values(0))

    # Use an excerpt of the overall DataFrame to assign the cluster info
    excerpt = potential_dict_nojoin[year].loc[potential_dict_nojoin[year]["cluster"].isna(),"cluster"].to_frame()
    excerpt = excerpt.reset_index().set_index("Prozess")
    excerpt["cluster"] = clusters_to_use["cluster"]
    excerpt = excerpt.set_index("sector", append=True)
    potential_dict_nojoin[year].loc[potential_dict_nojoin[year]["cluster"].isna(),"help_sector"] = "tcs+hoho"
    potential_dict_nojoin[year].loc[potential_dict_nojoin[year]["cluster"].isna(),"cluster"] = excerpt.loc[:,"cluster"]

In [ ]:
potential_dict["2050"].head()

In [ ]:
potential_dict["SQ"].head()

## Show and explore the clusters
Print the clusters in order to visually inspect them

In [ ]:
if print_clusters:
    for el in np.sort(potential_dict["SQ"]["cluster"].unique()):
        print(20 * "-")
        print("cluster number: "+str(el))
        print(20 * "-")
        display(potential_dict["SQ"][potential_dict["SQ"]["cluster"] == el])

## Create transposed potential data sets for combining it with availability data

In [ ]:
# Initialize dicts to store the transposed (and not joined) potential data
# as well as the availability time series
potential_T_dict = {}
potential_ind_T_dict = {}
potential_tcs_T_dict = {}
potential_hoho_T_dict = {}
potential_tcs_hoho_T_dict = {}

for year in years:
    # Create transposed subsets of potential_dict_nojoin
    potential_T_dict[year] = potential_dict_nojoin[year].T
    potential_ind_T_dict[year] = potential_dict_nojoin[year][
        potential_dict_nojoin[year].index.get_level_values(1) == "ind"].T
    potential_tcs_T_dict[year] = potential_dict_nojoin[year][
        potential_dict_nojoin[year].index.get_level_values(1) == "tcs"].T
    potential_hoho_T_dict[year] = potential_dict_nojoin[year][
        potential_dict_nojoin[year].index.get_level_values(1) == "hoho"].T
    potential_tcs_hoho_T_dict[year] = potential_dict_nojoin[year][
        potential_dict_nojoin[year].index.get_level_values(1) == "tcs+hoho"].T

In [ ]:
potential_T_dict["SQ"].head()

In [ ]:
potential_T_dict["2050"].head()

## Read in and match availability data

* Read in the availability time series data.
* Match the data to the respective demand response categories.
        
> __*Conceptual question:*__ 
> _Rethink, whether availability timeseries indexed by year do make sense and how this should be handled within (investment) model runs._
> * _Pros: Different years (e.g. temperature or weekday patterns) could be depicted._
> * _Cons: Some effort has to be made to achieve that_

In [ ]:
# Read in availability time series data
availability_ind_pos = pd.read_excel(path_folder_availability+filename_availability_in,
                                     sheet_name = "ind_pos", header=0, index_col=0)

availability_ind_neg = pd.read_excel(path_folder_availability+filename_availability_in,
                                     sheet_name = "ind_neg", header=0, index_col=0)

availability_tcs_pos = pd.read_excel(path_folder_availability+filename_availability_in,
                                     sheet_name = "tcs_pos", header=0, index_col=0)

availability_tcs_neg = pd.read_excel(path_folder_availability+filename_availability_in,
                                     sheet_name = "tcs_neg", header=0, index_col=0)

availability_hoho_pos = pd.read_excel(path_folder_availability+filename_availability_in,
                                     sheet_name = "hoho_pos", header=0, index_col=0)

availability_hoho_neg = pd.read_excel(path_folder_availability+filename_availability_in,
                                     sheet_name = "hoho_neg", header=0, index_col=0)

In [ ]:
# Map the demand response categories to the column names used
# for the availability time series (naming conventions from the bachelor theses are widely used).
availability_cats_ind_pos = {'normiertes\nLRP Alu': ('Primäraluminiumelektrolyse', 'ind'),
                            'normiertes\nLRP CHL': ('Chlor-Alkali-Elektrolyse', 'ind'),
                            'normiertes\nLRP HS': ('Holz- und Zellstoffherstellung', 'ind'),
                            'normiertes\nLRP PM': ('Papiermaschinen', 'ind'),
                            'normiertes\nLRP ES': ('Elektrostahlherstellung (Lichtbogenofen)', 'ind'),
                            'normiertes\nLRP RZM': ('Zementherstellung', 'ind'),
                            'normiertes\nLRP KUZI': ('Kupfer- und Zinkherstellung (Elektrolyse)', 'ind'),
                            'normiertes\nLRP AP': ('Altpapierrecycling (Pulper)', 'ind')}

availability_cats_ind_neg = {'normiertes\nLZP Alu': ('Primäraluminiumelektrolyse', 'ind'),
                            'normiertes\nLZP CHL': ('Chlor-Alkali-Elektrolyse', 'ind'),
                            'normiertes\nLZP HS': ('Holz- und Zellstoffherstellung', 'ind'),
                            'normiertes\nLZP PM': ('Papiermaschinen', 'ind'),
                            'normiertes\nLZP ES': ('Elektrostahlherstellung (Lichtbogenofen)', 'ind'),
                            'normiertes\nLZP RZM': ('Zementherstellung', 'ind'),
                            'normiertes\nLZP KUZI': ('Kupfer- und Zinkherstellung (Elektrolyse)', 'ind'),
                            'normiertes\nLZP AP': ('Altpapierrecycling (Pulper)', 'ind')}

# Note: Instead of KGR in the tcs sector, profiles for food retailing are introduced and used.
availability_cats_tcs_pos = {'Lastabschaltung LÜ normiert': ('Belüftung', 'tcs'), 
                            'Lastabschaltung WVP normiert': ('Pumpenanwendungen in der Wasserversorgung', 'tcs'),
                            'Lastabschaltung KGR normiert': ('Prozesskälte Handel', 'tcs'),
                            'Lastabschaltung KÜ normiert': ('Kühlhäuser', 'tcs'),
                            'Lastabschaltung KA normiert': ('Klimakälte', 'tcs'),  
                            'Lastabschaltung EH normiert': ('Nachtspeicherheizungen', 'tcs'),
                            'Lastabschaltung WP normiert': ('Wärmepumpen', 'tcs'), 
                            'Lastabschaltung WW normiert': ('Warmwasserbereitstellung', 'tcs')}

availability_cats_tcs_neg = {'Lastzuschaltung LÜ normiert': ('Belüftung', 'tcs'),
                             'Lastzuschaltung WVP normiert': ('Pumpenanwendungen in der Wasserversorgung', 'tcs'),
                             'Lastzuschaltung KGR normiert': ('Prozesskälte Handel', 'tcs'),
                             'Lastzuschaltung KÜ normiert': ('Kühlhäuser', 'tcs'),
                             'Lastzuschaltung KA normiert': ('Klimakälte', 'tcs'), 
                             'Lastzuschaltung EH normiert': ('Nachtspeicherheizungen', 'tcs'),
                             'Lastzuschaltung WP normiert': ('Wärmepumpen', 'tcs'), 
                             'Lastzuschaltung WW normiert': ('Warmwasserbereitstellung', 'tcs')}

availability_cats_hoho_pos = {'Lastabschaltung KGR': ('Kühlschränke', 'hoho'), 
                              'Lastabschaltung WM': ('Waschmaschinen', 'hoho'), 
                              'Lastabschaltung WT': ('Wäschetrockner', 'hoho'),
                              'Lastabschaltung GS': ('Geschirrspüler', 'hoho'), 
                              'Lastabschaltung NH normiert': ('Nachtspeicherheizungen', 'hoho'),
                              'Lastabschaltung WP normiert': ('Wärmepumpen', 'hoho'), 
                              'Lastabschaltung UP': ('Heizungsumwälzpumpen', 'hoho'),
                              'Lastabschaltung RK': ('Klimakälte', 'hoho'), 
                              'Lastabschaltung WW Tag': ('Warmwasserbereitstellung', 'hoho')}

availability_cats_hoho_neg = {'Lastzuschaltung KGR normiert': ('Kühlschränke', 'hoho'),
                              'Lastzuschaltung WM normiert': ('Waschmaschinen', 'hoho'),
                              'Lastzuschaltung WT normiert': ('Wäschetrockner', 'hoho'),
                              'Lastzuschaltung GS normiert': ('Geschirrspüler', 'hoho'), 
                              'Lastzuschaltung NH normiert': ('Nachtspeicherheizungen', 'hoho'),
                              'Lastzuschaltung WP normiert': ('Wärmepumpen', 'hoho'),
                              'Lastzuschaltung RK': ('Klimakälte', 'hoho'), 
                              'Lastzuschaltung WW normiert Tag': ('Warmwasserbereitstellung', 'hoho')}


# Change column name of availability time series DataFrame
availability_ind_pos = map_column_names(availability_ind_pos, availability_cats_ind_pos)
availability_ind_neg = map_column_names(availability_ind_neg, availability_cats_ind_neg)
availability_tcs_pos = map_column_names(availability_tcs_pos, availability_cats_tcs_pos)
availability_tcs_neg = map_column_names(availability_tcs_neg, availability_cats_tcs_neg)
availability_hoho_pos = map_column_names(availability_hoho_pos, availability_cats_hoho_pos)
availability_hoho_neg = map_column_names(availability_hoho_neg, availability_cats_hoho_neg)

> __*Important TODO for availability: Improve proxies for availability!*__

## Create availability time series for demand response categories for which availability info is missing

In general, there are three options for filling up the missing availability information:
1. Assign the existing availability time series of another process / demand response category due to large similarities (assumed).
2. Assign a constant availability profile for the entire year.
3. Create a synthetic load profile by defining availability factors for hours, weekdays and months and combining them.
    
The folowing availability information is used:
* Industry sector:
    * Foundries (German: "Gießereien") will be assigned the value for copper and zinc.
    * Calcium carbide will be assigned the value for electric furnace steel.
    * For the remaining categories, no profiles are available. As a first proxy, a constant availability profile is assumed.
* Tcs sector:
    * In the first place, for all categories not covered, a constant availability profile is assumed. _&rarr; Needs to be changed!_
* Household sector:
    * fride-freezer combinations as well as freezers will be assigned the same profile as fridges.
    * For all remaining categories not covered, a constant availability profile is assumed. _&rarr; Needs to be changed!_

### Use availability time series of existing categories

In [ ]:
# Boadcast values from existing demand response categories

# Industry sector
availability_ind_pos[('Calciumcarbid-Herstellung (Lichtbogenofen)', 'ind')] = \
    availability_ind_pos[('Elektrostahlherstellung (Lichtbogenofen)', 'ind')]
availability_ind_pos[('Gießereien (Induktionsofen)', 'ind')] = \
    availability_ind_pos[('Kupfer- und Zinkherstellung (Elektrolyse)', 'ind')]

availability_ind_neg[('Calciumcarbid-Herstellung (Lichtbogenofen)', 'ind')] = \
    availability_ind_neg[('Elektrostahlherstellung (Lichtbogenofen)', 'ind')]
availability_ind_neg[('Gießereien (Induktionsofen)', 'ind')] = \
    availability_ind_neg[('Kupfer- und Zinkherstellung (Elektrolyse)', 'ind')]

# tcs
availability_tcs_pos[('Prozesskälte', 'tcs')] =\
   availability_tcs_pos[('Kühlhäuser', 'tcs')] 
availability_tcs_neg[('Prozesskälte', 'tcs')] =\
   availability_tcs_neg[('Kühlhäuser', 'tcs')] 

# households
availability_hoho_pos[('Kühl- und Gefrierkombinationen', 'hoho')] = \
    availability_hoho_pos[('Kühlschränke', 'hoho')]
availability_hoho_pos[('Gefrierschränke und -truhen', 'hoho')] = \
    availability_hoho_pos[('Kühlschränke', 'hoho')]

availability_hoho_neg[('Kühl- und Gefrierkombinationen', 'hoho')] = \
    availability_hoho_neg[('Kühlschränke', 'hoho')]
availability_hoho_neg[('Gefrierschränke und -truhen', 'hoho')] = \
    availability_hoho_neg[('Kühlschränke', 'hoho')]

In [ ]:
# Determine for which categories availability time series are still missing
determine_missing_cols(potential_ind_T_dict["SQ"], availability_ind_pos, sector="industry (positive)")
determine_missing_cols(potential_ind_T_dict["SQ"], availability_ind_neg, sector="industry (negative)")
determine_missing_cols(potential_tcs_T_dict["SQ"], availability_tcs_pos, sector="tcs (positive)")
determine_missing_cols(potential_tcs_T_dict["SQ"], availability_tcs_neg, sector="tcs (negative)")
determine_missing_cols(potential_hoho_T_dict["SQ"], availability_hoho_pos, sector="hoho (positive)")
determine_missing_cols(potential_hoho_T_dict["SQ"], availability_hoho_neg, sector="hoho (negative)")

### Define synthetic profiles for availability
Define **synthetic profiles** for the remaining demand response categories missing availability information.
These comprise of a factor for the hourly, daily (weekday information) and monthly patterns. The three are multiplied in order to obtain hourly availability information. A very similar approach is used in Gils (2015).

> __*NOTE:*__
> * _Synthetic profiles used in the following are mostly **dummy values** to be replaced!!!_
> * _For a first "quick'n'dirty" model run, these are utilized, though._

In [ ]:
# Define synthetic profiles for the missing categories
# Create hours, weekday and month factors
hours_dict_pos = {}
days_dict_pos = {}
months_dict_pos = {}

hours_dict_neg = {}
days_dict_neg = {}
months_dict_neg = {}

In [ ]:
# Assign hourly profiles - positive
hours = range(0, 24)

# dummies
hours_dummy_constant = [1] * 24
hours_dummy_morning_evening_reduced = [0.8] * 6 + [1.0] * 12 + [0.8] * 6

# ind
hours_climate_cold_ind = [0.1, 0.1, 0.1, 0.2, 0.2, 0.2,
                          0.3, 0.4, 0.5, 0.6, 0.8, 1.0,
                          1.0, 1.0, 1.0, 1.0, 0.8, 0.7,
                          0.6, 0.5, 0.3, 0.2, 0.1, 0.1]

hours_dict_pos[('Klimakälte', 'ind')] = dict(zip(hours, hours_climate_cold_ind))
hours_dict_pos[('Prozesskälte', 'ind')] = dict(zip(hours, hours_dummy_constant))
hours_dict_pos[('Luftzerlegung', 'ind')] = dict(zip(hours, hours_dummy_constant))
hours_dict_pos[('Kühlung (Lebensmittelindustrie)', 'ind')] = dict(zip(hours, hours_dummy_constant))
hours_dict_pos[('Pumpenanwendungen', 'ind')] = dict(zip(hours, hours_dummy_morning_evening_reduced))
hours_dict_pos[('Beleuchtung', 'ind')] = dict(zip(hours, hours_dummy_morning_evening_reduced))
hours_dict_pos[('Druckluftanwendungen', 'ind')] = dict(zip(hours, hours_dummy_morning_evening_reduced))
hours_dict_pos[('Belüftung', 'ind')] = dict(zip(hours, hours_dummy_morning_evening_reduced))

# tcs
hours_ava_light_tcs = [0.05, 0.05, 0.1, 0.1, 0.15, 0.2, 
                       0.6, 1.0, 1.0, 0.9, 0.9, 0.8, 
                       0.8, 0.7, 0.7, 0.7, 0.6, 0.7, 
                       0.5, 0.2, 0.15, 0.15, 0.11, 0.05]

hours_dict_pos[('Prozesskälte', 'tcs')] = dict(zip(hours, hours_dummy_constant))
hours_dict_pos[('Beleuchtung', 'tcs')] = dict(zip(hours, hours_dummy_morning_evening_reduced))
hours_dict_pos[('Zerkleinerer', 'tcs')] = dict(zip(hours, hours_dummy_morning_evening_reduced))
hours_dict_pos[('Prozesswärme', 'tcs')] = dict(zip(hours, hours_dummy_morning_evening_reduced))
hours_dict_pos[('Abwasserbehandlung', 'tcs')] = dict(zip(hours, hours_dummy_morning_evening_reduced))

# hoho
hours_dict_pos[('Heizungsumwälzpumpen', 'hoho')] = dict(zip(hours, hours_dummy_morning_evening_reduced))

In [ ]:
# Assign hourly profiles - negative
hours = range(0, 24)

# dummies
hours_dummy_constant = [1] * 24
hours_dummy_morning_evening_reduced = [1] * 6 + [0.8] * 12 + [1] * 6

# ind
hours_climate_cold_ind = [0.05, 0.05, 0.05, 0.1, 0.1, 0.1,
                          0.15, 0.2, 0.2, 0.2, 0.2, 0,
                          0, 0, 0, 0, 0.2, 0.2,
                          0.2, 0.2, 0.15, 0.1, 0.05, 0.05]

hours_dict_neg[('Klimakälte', 'ind')] = dict(zip(hours, hours_climate_cold_ind))
hours_dict_neg[('Prozesskälte', 'ind')] = dict(zip(hours, hours_dummy_constant))
hours_dict_neg[('Luftzerlegung', 'ind')] = dict(zip(hours, hours_dummy_constant))
hours_dict_neg[('Kühlung (Lebensmittelindustrie)', 'ind')] = dict(zip(hours, hours_dummy_constant))
hours_dict_neg[('Pumpenanwendungen', 'ind')] = dict(zip(hours, hours_dummy_morning_evening_reduced))
hours_dict_neg[('Beleuchtung', 'ind')] = dict(zip(hours, hours_dummy_morning_evening_reduced))
hours_dict_neg[('Druckluftanwendungen', 'ind')] = dict(zip(hours, hours_dummy_morning_evening_reduced))
hours_dict_neg[('Belüftung', 'ind')] = dict(zip(hours, hours_dummy_morning_evening_reduced))

# tcs
hours_ava_light_tcs = [0.05, 0.05, 0.1, 0.1, 0.15, 0.2, 
                       0.2, 0, 0, 0.1, 0.1, 0.2, 
                       0.2, 0.2, 0.2, 0.2, 0.15, 0.1, 
                       0.1, 0.1, 0.1, 0.1, 0.05, 0.05]

hours_dict_neg[('Prozesskälte', 'tcs')] = dict(zip(hours, hours_dummy_constant))
hours_dict_neg[('Beleuchtung', 'tcs')] = dict(zip(hours, hours_dummy_morning_evening_reduced))
hours_dict_neg[('Zerkleinerer', 'tcs')] = dict(zip(hours, hours_dummy_morning_evening_reduced))
hours_dict_neg[('Prozesswärme', 'tcs')] = dict(zip(hours, hours_dummy_morning_evening_reduced))
hours_dict_neg[('Abwasserbehandlung', 'tcs')] = dict(zip(hours, hours_dummy_morning_evening_reduced))

# hoho
hours_dict_neg[('Heizungsumwälzpumpen', 'hoho')] = dict(zip(hours, hours_dummy_morning_evening_reduced))

In [ ]:
# Assign weekday profiles - positive
days = range(0, 7)

# dummies
days_ava_dummy_week_constant = [1] * 7
days_ava_dummy_weekend_slightly_reduced = [1] * 5 + [0.9, 0.75]
days_ava_dummy_weekend_reduced = [1] * 5 + [0.5, 0.05]

# ind
days_dict_pos[('Klimakälte', 'ind')] = dict(zip(days, days_ava_dummy_weekend_slightly_reduced))
days_dict_pos[('Prozesskälte', 'ind')] = dict(zip(days, days_ava_dummy_week_constant))
days_dict_pos[('Luftzerlegung', 'ind')] = dict(zip(days, days_ava_dummy_week_constant))
days_dict_pos[('Kühlung (Lebensmittelindustrie)', 'ind')] = dict(zip(days, days_ava_dummy_weekend_slightly_reduced))
days_dict_pos[('Pumpenanwendungen', 'ind')] = dict(zip(days, days_ava_dummy_weekend_slightly_reduced))
days_dict_pos[('Beleuchtung', 'ind')] = dict(zip(days, days_ava_dummy_weekend_slightly_reduced))
days_dict_pos[('Druckluftanwendungen', 'ind')] = dict(zip(days, days_ava_dummy_weekend_slightly_reduced))
days_dict_pos[('Belüftung', 'ind')] = dict(zip(days, days_ava_dummy_weekend_slightly_reduced))

# tcs
days_dict_pos[('Prozesskälte', 'tcs')] = dict(zip(days, days_ava_dummy_weekend_reduced))
days_dict_pos[('Beleuchtung', 'tcs')] = dict(zip(days, days_ava_dummy_weekend_reduced))
days_dict_pos[('Zerkleinerer', 'tcs')] = dict(zip(days, days_ava_dummy_weekend_slightly_reduced))
days_dict_pos[('Prozesswärme', 'tcs')] = dict(zip(days, days_ava_dummy_weekend_reduced))
days_dict_pos[('Abwasserbehandlung', 'tcs')] = dict(zip(days, days_ava_dummy_weekend_slightly_reduced))

# tcs
days_dict_pos[('Heizungsumwälzpumpen', 'hoho')] = dict(zip(days, days_ava_dummy_week_constant))

In [ ]:
# Assign weekday profiles - negative
days = range(0, 7)

# dummies
days_ava_dummy_week_constant = [1] * 7
days_ava_dummy_weekend_slightly_reduced = [0.5] * 5 + [0.8, 1]
days_ava_dummy_weekend_reduced = [0.2] * 5 + [0.5, 1]

# ind
days_dict_neg[('Klimakälte', 'ind')] = dict(zip(days, days_ava_dummy_weekend_slightly_reduced))
days_dict_neg[('Prozesskälte', 'ind')] = dict(zip(days, days_ava_dummy_week_constant))
days_dict_neg[('Luftzerlegung', 'ind')] = dict(zip(days, days_ava_dummy_week_constant))
days_dict_neg[('Kühlung (Lebensmittelindustrie)', 'ind')] = dict(zip(days, days_ava_dummy_weekend_slightly_reduced))
days_dict_neg[('Pumpenanwendungen', 'ind')] = dict(zip(days, days_ava_dummy_weekend_slightly_reduced))
days_dict_neg[('Beleuchtung', 'ind')] = dict(zip(days, days_ava_dummy_weekend_slightly_reduced))
days_dict_neg[('Druckluftanwendungen', 'ind')] = dict(zip(days, days_ava_dummy_weekend_slightly_reduced))
days_dict_neg[('Belüftung', 'ind')] = dict(zip(days, days_ava_dummy_weekend_slightly_reduced))

# tcs
days_dict_neg[('Prozesskälte', 'tcs')] = dict(zip(days, days_ava_dummy_weekend_reduced))
days_dict_neg[('Beleuchtung', 'tcs')] = dict(zip(days, days_ava_dummy_weekend_reduced))
days_dict_neg[('Zerkleinerer', 'tcs')] = dict(zip(days, days_ava_dummy_weekend_slightly_reduced))
days_dict_neg[('Prozesswärme', 'tcs')] = dict(zip(days, days_ava_dummy_weekend_reduced))
days_dict_neg[('Abwasserbehandlung', 'tcs')] = dict(zip(days, days_ava_dummy_weekend_slightly_reduced))

# tcs
days_dict_neg[('Heizungsumwälzpumpen', 'hoho')] = dict(zip(days, days_ava_dummy_week_constant))

In [ ]:
# Assign months profiles - positive
months = range(1, 13)

# dummies
months_ava_dummy_constant = [1] * 12
months_ava_dummy_cold_seasons = [1, 0.9, 0.8, 0.7, 0.5, 0.25, 
                                0.25, 0.25, 0.5, 0.7, 0.9, 1]
months_ava_dummy_warm_seasons = [0] * 4 + [0.3, 0.7] + [1] * 2 + [0.6, 0.1] + [0] * 2
month_ava_heating_seasons = [1, 0.8, 0.5, 0.1] + 5 * [0] + [0.2, 0.5, 1]

# ind
months_dict_pos[('Klimakälte', 'ind')] = dict(zip(months, months_ava_dummy_warm_seasons))
months_dict_pos[('Prozesskälte', 'ind')] = dict(zip(months, months_ava_dummy_constant))
months_dict_pos[('Luftzerlegung', 'ind')] = dict(zip(months, months_ava_dummy_constant))
months_dict_pos[('Kühlung (Lebensmittelindustrie)', 'ind')] = dict(zip(months, months_ava_dummy_constant))
months_dict_pos[('Pumpenanwendungen', 'ind')] = dict(zip(months, months_ava_dummy_constant))
months_dict_pos[('Beleuchtung', 'ind')] = dict(zip(months, months_ava_dummy_cold_seasons))
months_dict_pos[('Druckluftanwendungen', 'ind')] = dict(zip(months, months_ava_dummy_constant))
months_dict_pos[('Belüftung', 'ind')] = dict(zip(months, months_ava_dummy_constant))

# tcs
months_dict_pos[('Prozesskälte', 'tcs')] = dict(zip(months, months_ava_dummy_constant))
months_dict_pos[('Beleuchtung', 'tcs')] = dict(zip(months, months_ava_dummy_cold_seasons))
months_dict_pos[('Zerkleinerer', 'tcs')] = dict(zip(months, months_ava_dummy_constant))
months_dict_pos[('Prozesswärme', 'tcs')] = dict(zip(months, months_ava_dummy_constant))
months_dict_pos[('Abwasserbehandlung', 'tcs')] = dict(zip(months, months_ava_dummy_constant))

# tcs
months_dict_pos[('Heizungsumwälzpumpen', 'hoho')] = dict(zip(months, month_ava_heating_seasons))

In [ ]:
# Assign months profiles - negative
months = range(1, 13)

# dummies
months_ava_dummy_constant = [1] * 12
months_ava_dummy_cold_seasons = [0.2, 0.4, 0.5, 0.6, 1, 0.4, 
                                0.4, 0.5, 1, 0.7, 0.5, 0.2]
months_ava_dummy_warm_seasons = [0] * 4 + [0.2, 0.4] + [0.2] * 2 + [1, 0.2] + [0] * 2
month_ava_heating_seasons = [0.4, 0.7, 1, 0.4] + 5 * [0] + [0.4, 1, 0.4]

# ind
months_dict_neg[('Klimakälte', 'ind')] = dict(zip(months, months_ava_dummy_warm_seasons))
months_dict_neg[('Prozesskälte', 'ind')] = dict(zip(months, months_ava_dummy_constant))
months_dict_neg[('Luftzerlegung', 'ind')] = dict(zip(months, months_ava_dummy_constant))
months_dict_neg[('Kühlung (Lebensmittelindustrie)', 'ind')] = dict(zip(months, months_ava_dummy_constant))
months_dict_neg[('Pumpenanwendungen', 'ind')] = dict(zip(months, months_ava_dummy_constant))
months_dict_neg[('Beleuchtung', 'ind')] = dict(zip(months, months_ava_dummy_cold_seasons))
months_dict_neg[('Druckluftanwendungen', 'ind')] = dict(zip(months, months_ava_dummy_constant))
months_dict_neg[('Belüftung', 'ind')] = dict(zip(months, months_ava_dummy_constant))

# tcs
months_dict_neg[('Prozesskälte', 'tcs')] = dict(zip(months, months_ava_dummy_constant))
months_dict_neg[('Beleuchtung', 'tcs')] = dict(zip(months, months_ava_dummy_cold_seasons))
months_dict_neg[('Zerkleinerer', 'tcs')] = dict(zip(months, months_ava_dummy_constant))
months_dict_neg[('Prozesswärme', 'tcs')] = dict(zip(months, months_ava_dummy_constant))
months_dict_neg[('Abwasserbehandlung', 'tcs')] = dict(zip(months, months_ava_dummy_constant))

# tcs
months_dict_neg[('Heizungsumwälzpumpen', 'hoho')] = dict(zip(months, month_ava_heating_seasons))

In [ ]:
# Fill missing availability time series
availability_ind_pos = assign_availability_remaining(
    potential_ind_T_dict["SQ"], availability_ind_pos,
    synthetic_cols=[('Klimakälte', 'ind'), ('Prozesskälte', 'ind'), 
                    ('Luftzerlegung', 'ind'), 
                    ('Kühlung (Lebensmittelindustrie)', 'ind'), 
                    ('Pumpenanwendungen', 'ind'), ('Beleuchtung', 'ind'), 
                    ('Druckluftanwendungen', 'ind'), ('Belüftung', 'ind')],
    hours_dict = hours_dict_pos,
    days_dict = days_dict_pos,
    months_dict = months_dict_pos)

availability_ind_neg = assign_availability_remaining(
    potential_ind_T_dict["SQ"], availability_ind_neg,
    synthetic_cols=[('Klimakälte', 'ind'), ('Prozesskälte', 'ind'), 
                    ('Luftzerlegung', 'ind'), 
                    ('Kühlung (Lebensmittelindustrie)', 'ind'), 
                    ('Pumpenanwendungen', 'ind'), ('Beleuchtung', 'ind'), 
                    ('Druckluftanwendungen', 'ind'), ('Belüftung', 'ind')],
    hours_dict = hours_dict_neg,
    days_dict = days_dict_neg,
    months_dict = months_dict_neg)

availability_tcs_pos = assign_availability_remaining(
    potential_tcs_T_dict["SQ"], availability_tcs_pos,
    synthetic_cols=[('Prozesskälte', 'tcs'), ('Beleuchtung', 'tcs'), 
                    ('Zerkleinerer', 'tcs'), ('Prozesswärme', 'tcs'), 
                    ('Abwasserbehandlung', 'tcs')],
    hours_dict = hours_dict_pos,
    days_dict = days_dict_pos,
    months_dict = months_dict_pos)

availability_tcs_neg = assign_availability_remaining(
    potential_tcs_T_dict["SQ"], availability_tcs_neg,
    synthetic_cols=[('Prozesskälte', 'tcs'), ('Beleuchtung', 'tcs'), 
                    ('Zerkleinerer', 'tcs'), ('Prozesswärme', 'tcs'), 
                    ('Abwasserbehandlung', 'tcs')],
    hours_dict = hours_dict_neg,
    days_dict = days_dict_neg,
    months_dict = months_dict_neg)

# No missing availability for hoho pos -> skip!

availability_hoho_neg = assign_availability_remaining(
    potential_hoho_T_dict["SQ"], availability_hoho_neg,
    synthetic_cols=[('Heizungsumwälzpumpen', 'hoho')],
    hours_dict = hours_dict_neg,
    days_dict = days_dict_neg,
    months_dict = months_dict_neg)

## Create load profiles per cluster
Create load profiles for the demand response categories.
In principle, two approaches could be used here:
1. Most simple proxy: Use availability time series in positive direction. &rarr; drawback: very rough estimate
2. More advanced: Use profiles per WZ as an intermediate product of the the demand regio disaggregator tool.

Profiles are scaled with installed capacity in both cases. Simultaneity factors are introduced to account for the fact that not all demands happen simultaneously.

### Preparation for load profile assessment
* Provide availability time series with cluster information
* Initialize empty dictionaries to store the time series
* Determine which max. simultaneity factors to use &rarr; assumption is needed as long as there is no information on energy consumption or full load hours
* Copy availability time series in order to be able to use it for both approaches

In [ ]:
# Assign cluster information
availability_ind_pos.loc["cluster"] = potential_ind_T_dict["SQ"].loc["cluster"]
availability_ind_neg.loc["cluster"] = potential_ind_T_dict["SQ"].loc["cluster"]
availability_tcs_pos.loc["cluster"] = potential_tcs_T_dict["SQ"].loc["cluster"]
availability_tcs_neg.loc["cluster"] = potential_tcs_T_dict["SQ"].loc["cluster"]
availability_hoho_pos.loc["cluster"] = potential_hoho_T_dict["SQ"].loc["cluster"]
availability_hoho_neg.loc["cluster"] = potential_hoho_T_dict["SQ"].loc["cluster"]

In [ ]:
ind_ts_dict = {}
tcs_ts_dict = {}
hoho_ts_dict = {}
overall_ts_dict = {}

# determine maximum simultaneity factors per sector
max_simultaneity_ind = 0.9
max_simultaneity_tcs = 0.4
max_simultaneity_hoho = 0.1

In [ ]:
# Create a copy of availability time series
load_profile_ind_pos = availability_ind_pos.copy()
load_profile_tcs_pos = availability_tcs_pos.copy()
load_profile_hoho_pos = availability_hoho_pos.copy()

### Approach 1: Use data from the demand regio disaggregator
* Read in normalized profiles as output from the disaggregator tool
* Map the demand response profiles with the WZs used in the disaggregator tool (hard-coded)
* For the WZs which can be directly assigned, the respective demand pattern is used
* For cross-cutting technologies no data exists, hence, the availability time series is used as a backup

In [ ]:
if not use_ava_ts_for_profiles:
    profiles_ind_normalized = pd.read_csv("profiles_industry_normalized.csv", 
                                          index_col=0, parse_dates=True)
    profiles_tcs_normalized = pd.read_csv("profiles_cts_normalized.csv", 
                                          index_col=0, parse_dates=True)
    profiles_hoho_normalized = pd.read_csv("profiles_households_normalized.csv",
                                           index_col=0, parse_dates=True)

    WZ_mapping = pd.read_excel("remaining_categories_WZ_mapping.xlsx", index_col=[0, 1])

    profiles_ind_normalized.columns = profiles_ind_normalized.columns.astype(int)
    profiles_tcs_normalized.columns = profiles_tcs_normalized.columns.astype(int)
    profiles_hoho_normalized.columns = profiles_hoho_normalized.columns.astype(int)
    
    # Prepare WZ mapping
    WZ_mapping.index.names = ["Prozess", "help_sector"]
    WZ_mapping["sector"] = np.where(WZ_mapping.index.get_level_values(1) == "tcs+hoho", "hoho",
                                    WZ_mapping.index.get_level_values(1))
    
    WZ_mapping = WZ_mapping.reset_index(level=1)
    WZ_mapping = WZ_mapping.set_index("sector", append=True)

    WZs_used = WZ_mapping["WZ"].unique()
    print(WZs_used)
    
    display(WZ_mapping.head(10))

In [ ]:
if not use_ava_ts_for_profiles:
    
    ind_ts_df = pd.DataFrame(index=profiles_ind_normalized.index,
                         columns=potential_ind_T_dict["SQ"].columns)
    tcs_ts_df = pd.DataFrame(index=profiles_tcs_normalized.index,
                             columns=potential_tcs_T_dict["SQ"].columns)
    hoho_ts_df = pd.DataFrame(index=profiles_hoho_normalized.index,
                             columns=potential_hoho_T_dict["SQ"].columns)

    cols_used_ind = []
    cols_used_tcs = []
    cols_used_hoho = []

    remaining_ts_cats = []
    
    for WZ in WZs_used:
        ix = [list(el) for el in WZ_mapping[WZ_mapping["WZ"] == WZ].index]
        
        if WZ in profiles_ind_normalized.columns:
            ind_ts_df.loc[:,ix] = np.transpose([profiles_ind_normalized[WZ].values] * len(ix))
            cols_used_ind.extend(ix)

        elif WZ in profiles_tcs_normalized.columns:
            tcs_ts_df.loc[:,ix] = np.transpose([profiles_tcs_normalized[WZ].values] * len(ix))
            cols_used_tcs.extend(ix)

        elif WZ in profiles_hoho_normalized.columns:
            hoho_ts_df.loc[:,ix] = np.transpose([profiles_hoho_normalized[WZ].values] * len(ix))
            cols_used_hoho.extend(ix)

        else:
            remaining_ts_cats.extend(ix)

    overall_ts_df = pd.concat([ind_ts_df, 
                               tcs_ts_df,
                               hoho_ts_df], axis=1)
    
    # Replace the default load time series with these values
    load_profile_ind_pos.loc[:, cols_used_ind] = ind_ts_df.loc[:, cols_used_ind]
    load_profile_tcs_pos.loc[:, cols_used_tcs] = tcs_ts_df.loc[:, cols_used_tcs]
    load_profile_hoho_pos.loc[:, cols_used_hoho] = hoho_ts_df.loc[:, cols_used_hoho]
    
    # Add cluster info again
    load_profile_ind_pos.loc["cluster"] = availability_ind_pos.loc["cluster"]
    load_profile_tcs_pos.loc["cluster"] = availability_tcs_pos.loc["cluster"]
    load_profile_hoho_pos.loc["cluster"] = availability_hoho_pos.loc["cluster"]
    
    # Determine cols that need a special treatment,
    # i.e. columns for which the availability time series will be used
    #remaining_ts_cats = list(set(potential_dict["SQ"].index) - (set(overall_ts_df.columns)))
    print(f"Using availability time series information for categories: \n{remaining_ts_cats}")

### Approach 2: Use availability time series as simplest proxy.
> _NOTE: If approach one was chosen, this will be run as well, but the data used will differ!_
* Multiply availability time series with installed capacity and maximum simultaneity factor for the respective sector
* Include cluster information for grouping.
* Combine to an overall load profile time series dataset which will be grouped within the clusters

In [ ]:
for year in years:
    for col in cols:
        # industry        
        inst_cap_ind = potential_ind_T_dict[year].loc[col+"_installed_cap"]
        ind_ts_dict[(year, col)] = pd.DataFrame(index=load_profile_ind_pos.index, 
                                                columns=load_profile_ind_pos.columns)
        ind_ts_dict[(year, col)].iloc[:-1] = load_profile_ind_pos.iloc[:-1].mul(
            inst_cap_ind).mul(max_simultaneity_ind)
        ind_ts_dict[(year, col)].loc["cluster"] = load_profile_ind_pos.loc["cluster"]

        # tcs
        inst_cap_tcs = potential_tcs_T_dict[year].loc[col+"_installed_cap"]
        tcs_ts_dict[(year, col)] = pd.DataFrame(index=load_profile_tcs_pos.index, 
                                                columns=load_profile_tcs_pos.columns)
        tcs_ts_dict[(year, col)].iloc[:-1] = load_profile_tcs_pos.iloc[:-1].mul(
            inst_cap_tcs).mul(max_simultaneity_tcs)
        tcs_ts_dict[(year, col)].loc["cluster"] = load_profile_tcs_pos.loc["cluster"]

        # hoho
        inst_cap_hoho = potential_hoho_T_dict[year].loc[col+"_installed_cap"]
        hoho_ts_dict[(year, col)] = pd.DataFrame(index=load_profile_hoho_pos.index, 
                                                 columns=load_profile_hoho_pos.columns)
        hoho_ts_dict[(year, col)].iloc[:-1] = load_profile_hoho_pos.iloc[:-1].mul(
            inst_cap_hoho).mul(max_simultaneity_hoho)
        hoho_ts_dict[(year, col)].loc["cluster"] = load_profile_hoho_pos.loc["cluster"]

        overall_ts_dict[(year, col)] = pd.concat([ind_ts_dict[(year, col)], 
                                                  tcs_ts_dict[(year, col)], 
                                                  hoho_ts_dict[(year, col)]], axis=1)

In [ ]:
tcs_ts_dict[("SQ","50%")].tail()

### Visualize load profiles

In [ ]:
ind_ts_dict[("SQ", "50%")].iloc[:-1].plot()
plt.ylim([0,2500])
plt.legend(bbox_to_anchor=(1.05, 1))
plt.show()

In [ ]:
tcs_ts_dict[("SQ", "50%")].iloc[:-1].plot()
plt.ylim([0,2500])
plt.legend(bbox_to_anchor=(1.05, 1))
plt.show()

In [ ]:
hoho_ts_dict[("SQ", "50%")].iloc[:-1].plot()
plt.ylim([0,10000])
plt.legend(bbox_to_anchor=(1.05, 1))
plt.show()

##  Create availability time series per cluster
* Calculate weighted averages within clusters.
* Save the results, i.e. availability time series per cluster, to a csv file.

In [ ]:
sector_dict = {}

for year in years:

    sector_dict[year] = {"ind": (availability_ind_pos, availability_ind_neg, potential_ind_T_dict[year]),
                       "tcs": (availability_tcs_pos, availability_tcs_neg, potential_tcs_T_dict[year]),
                       "hoho": (availability_hoho_pos, availability_hoho_neg, potential_hoho_T_dict[year])}

**Interpretation of availability factors per cluster**:
* Maximum availability per cluster does not necessarily have to be 1.
* A value of 0.88 implies that at maximum 88% of the overall cluster capacity are available at the same time. &rarr; Similar interpretation than simultaneity factor.
* Nevertheless, a rescaling is done here for the sake of easier interpretation. &rarr; I.e. maximum values will be 1.0 and cluster capacity is adapted accordingly.

In [ ]:
np.sort(potential_T_dict["SQ"].loc["cluster"].unique())

In [ ]:
# Extract info on combined clusters for tcs and hoho separately in order not to overwrite it
tcs_hoho_clusters = potential_dict["SQ"].loc[potential_dict["SQ"].index.get_level_values(1) == "tcs+hoho", 
                                             "cluster"].unique()

**_TODO: Check assignments here! Seems a bit buggy / inconvenient!_**

In [ ]:
pot_cols = ["potential_pos_overall", "potential_neg_overall", "installed_cap"]
cols_of_interest = [i+"_"+j for i in cols for j in pot_cols]

# Use dicts and DataFrames to store overall cluster potential and availability time series per cluster
cluster_overall_pot_dict = {}
cluster_overall_ts_dict = {}
availability_clusters = pd.DataFrame(index=availability_ind_pos.index).drop("cluster")

# Create availability time series within clusters by calculating weighted averages
for year in years:
    
    cluster_overall_pot_df = pd.DataFrame(columns=cols_of_interest)
    load_timeseries = pd.DataFrame(index=overall_ts_dict[("SQ","50%")].iloc[:-1].index)
    
    for el in np.sort(potential_T_dict["SQ"].loc["cluster"].unique()):
        
        to_concat = []
        cluster_data = {}
        
        for col in cols:
            sector = potential_T_dict[year].loc[:,potential_T_dict[year].loc[
                "cluster"] == el].columns.get_level_values(1)[0]
            
            # introduce shortcuts for readability
            org_ava_pos = sector_dict[year][sector][0]
            org_ava_neg = sector_dict[year][sector][1]
            pots_T = sector_dict[year][sector][2]
            
            # Calculate a weighted average for positive potentials (weights: maximum potential information)
            ava_pos = org_ava_pos.loc[:,org_ava_pos.loc["cluster"] == el].drop("cluster")    
            pot_pos = pots_T.loc[col+"_potential_pos_overall", pots_T.loc["cluster"] == el]
            overall_pot_pos = pot_pos.sum()
            
            if overall_pot_pos != 0:
                ava_pos[col+"_pos_cluster_"+str(el)] = ava_pos.mul(pot_pos).div(overall_pot_pos).sum(axis=1)

                # Do rescaling: Adjust (reduce) potential information if necessary 
                # and scale max value of availability time series to 1            
                max_pot_pos = ava_pos[col+"_pos_cluster_"+str(el)].max() * overall_pot_pos
                ava_pos[col+"_pos_cluster_"+str(el)] = \
                    ava_pos[col+"_pos_cluster_"+str(el)].div(ava_pos[col+"_pos_cluster_"+str(el)].max())
            
            # Calculate a weighted average for negative potentials (weights: potential information)
            ava_neg = org_ava_neg.loc[:,org_ava_neg.loc["cluster"] == el].drop("cluster")    
            pot_neg = pots_T.loc[col+"_potential_neg_overall", pots_T.loc["cluster"] == el]
            overall_pot_neg = pot_neg.sum()
            
            if overall_pot_neg != 0:
                ava_neg[col+"_neg_cluster_"+str(el)] = ava_neg.mul(pot_neg).div(overall_pot_neg).sum(axis=1)

                # Do rescaling: Adjust (reduce) potential information if necessary 
                # and scale max value of availability time series to 1
                max_pot_neg = ava_neg[col+"_neg_cluster_"+str(el)].max() * overall_pot_neg
                ava_neg[col+"_neg_cluster_"+str(el)] = \
                    ava_neg[col+"_neg_cluster_"+str(el)].div(ava_neg[col+"_neg_cluster_"+str(el)].max())

            if el not in tcs_hoho_clusters:
                cluster_label = sector+"_cluster-"+str(int(el))
            else:
                cluster_label = "tcs+hoho_cluster-"+str(int(el))

            # show exemplarily availability time series for clusters
            if year == "SQ":
                availability_clusters[col+"_"+cluster_label+"_pos"] = ava_pos[col+"_pos_cluster_"+str(el)]
                availability_clusters[col+"_"+cluster_label+"_neg"] = ava_neg[col+"_neg_cluster_"+str(el)]
                availability_clusters = availability_clusters.round(4)
            
                if col == "50%":
                    # Show a sample week for the clusters (SQ, positive potentials and median only)
                    fig, ax = plt.subplots(figsize=(10,5))
                    _ = ava_pos.iloc[:168,:-1].plot(ax=ax)
                    _ = ava_pos.iloc[:168,-1:].plot(ax=ax, linewidth=5)
                    _ = plt.title("Cluster "+str(el))
                    _ = plt.legend(bbox_to_anchor=(1.05, 1))
                    plt.show()
                    fig.savefig("cluster_"+str(el)+".png")
                    plt.close()
            
            # Save potential outputs
            cluster_data[col+"_"+pot_cols[0]] = max_pot_pos
            cluster_data[col+"_"+pot_cols[1]] = max_pot_neg
            
            # Assign load profiles and store them in a dict
            load_timeseries[col+"_"+cluster_label] = overall_ts_dict[(year, col)].loc[
                :,overall_ts_dict[(year, col)].loc["cluster"] == el].sum(axis=1).drop("cluster")
            
            max_cap = load_timeseries[col+"_"+cluster_label].max()
            
            cluster_data[col+"_"+pot_cols[2]] = max_cap
            
        # Combine potential outputs and store them in dict of DataFrames
        cluster_overall_pot_df.loc[cluster_label, [i+"_"+j for i in cols 
                                                           for j in pot_cols]] = cluster_data

        cluster_overall_pot_dict[year] = cluster_overall_pot_df
    
        cluster_overall_ts_dict[year] = load_timeseries

display(availability_clusters.tail())

Introduce some fixes:
* Normalize load profiles again in order to be able to use them combined with maximum capacity demand
* Rename installed\_cap to max\_cap in order to prevent misinterpretation

In [ ]:
# Normalize load profiles (for usage in oemof)
# Rename column to max cap in order to be able to distinct it from installed capacity
for year in years:
    cluster_overall_ts_dict[year] = cluster_overall_ts_dict[year].div(cluster_overall_ts_dict[year].max())
    for col in cols:
        cluster_overall_pot_dict[year].rename({col + "_installed_cap": col + "_max_cap" for col in cols},
                                      axis=1, inplace=True)

In [ ]:
cluster_overall_pot_dict["SQ"].head()

In [ ]:
cluster_overall_pot_dict["2050"].head()

In [ ]:
cluster_overall_ts_dict["2050"].max()

In [ ]:
cluster_labels = cluster_overall_pot_dict["SQ"].index.values
cluster_ts_by_cols = {}
ava_pos_ts_by_cols = {}
ava_neg_ts_by_cols = {}

# Split timeseries into subsets
for col in cols:
    for year in years:
        ava_cols = [i+"_"+j for i in [col] for j in cluster_labels]
        ava_cols_pos = [i+"_"+j+"_pos" for i in [col] for j in cluster_labels]
        ava_cols_neg = [i+"_"+j+"_neg" for i in [col] for j in cluster_labels]

        cluster_ts_by_cols[col+"_"+year] = cluster_overall_ts_dict[year][ava_cols]
        ava_pos_ts_by_cols[col+"_"+year] = availability_clusters[ava_cols_pos]
        ava_neg_ts_by_cols[col+"_"+year] = availability_clusters[ava_cols_neg]

In [ ]:
cluster_ts_by_cols["25%_SQ"]

In [ ]:
if write_outputs:
    availability_clusters.to_csv(path_folder_availability+filename_availability_out, sep=";", decimal=",")
    write_multiple_sheets(cluster_ts_by_cols, path_folder_parameterization, 
                          filename_load_profiles_out+".xlsx")

## Group the data within the clusters and write outputs
Determine, how the grouping will take place and which aggregation rules to apply for a parameter:
* *grouping_cols*: Columns to group by (cluster numbers)
* *mean_cols*: Columns for which the weighted average is used
* *sum_cols*: Columns for which entries are summed up

The average positive potential is used for calculating weigthed averages.<br>
> _Note: Again, there are possible alternatives here:_
> * _Use the installed capacity &rarr; drawback: missing for most demand response categories._
> * _Use positive or negative capacity dependent on parameter. &rarr; drawback: may lead to inconsistencies due to differing weights._

In [ ]:
# Extract parameters of interest for further model-based analyses
params_of_interest_all = params_tbu + params_remaining
params_of_interest_all.remove("year_key")
params_of_interest = set()
for el in params_of_interest_all:
    params_of_interest.add(el[4:])
params_of_interest = list(params_of_interest)

In [ ]:
grouping_cols = ["cluster"]
other_cols = []

mean_list = []
sum_list = []
for k, v in parameters_groups_dict.items():
    if k not in params_of_interest:
        continue
    if v[0] == "mean":
        mean_list.append(k)
    elif v[0] == "sum":
        sum_list.append(k)
        
mean_cols = create_parameter_combinations(mean_list, cols)
sum_cols = create_parameter_combinations(sum_list, cols)

In [ ]:
# Renaming of cluster identifiers
name_dict = {"ind_cluster-2112": "ind_cluster-1",
             "ind_cluster-2113": "ind_cluster-2",
             "tcs_cluster-12": "tcs_cluster",
             "hoho_cluster-2": "hoho_cluster",
             "tcs+hoho_cluster-112": "tcs_hoho_cluster"}

In [ ]:
def round_of_rating(number):
    """Round a number to the closest quarter integer. """
    return round(number * 4) / 4

In [ ]:
# Calculate the parameters for the clusters
ind_dict = {}
tcs_dict = {}
hoho_dict = {}
tcs_hoho_dict = {}
overall_dict = {}

pot_cols = ["potential_pos_overall", "potential_neg_overall", "max_cap"]
dur_cols = ["interference_duration_neg", "interference_duration_pos", 
            "regeneration_duration", "shifting_duration"]
cost_cols = ["fixed_costs", "variable_costs", "specific_investments"]
other_cols = ["maximum_activations_year", "installed_cap", "max_cap",
              "potential_neg_overall", "potential_pos_overall"]

for col in cols:
    for year in years:
        ind_dict[col+"_"+year] = group_potential(potential_dict[year][potential_dict[year].index.get_level_values(1) == "ind"], 
                                                 grouping_cols, weight=col+"_potential_pos_overall", mean_cols=mean_cols, 
                                                 sum_cols=sum_cols, other_cols=other_cols,
                                                 sector="ind", drop=[el for el in cols if el != col]).round(2)
        
        tcs_dict[col+"_"+year] = group_potential(potential_dict[year][potential_dict[year].index.get_level_values(1) == "tcs"], 
                                                 grouping_cols, weight=col+"_potential_pos_overall", mean_cols=mean_cols, 
                                                 sum_cols=sum_cols, other_cols=other_cols,
                                                 sector="tcs", drop=[el for el in cols if el != col]).round(2)
        
        hoho_dict[col+"_"+year] = group_potential(potential_dict[year][potential_dict[year].index.get_level_values(1) == "hoho"], 
                                                  grouping_cols, weight=col+"_potential_pos_overall", mean_cols=mean_cols, 
                                                  sum_cols=sum_cols,  other_cols=other_cols,
                                                  sector="hoho", drop=[el for el in cols if el != col]).round(2)
        
        tcs_hoho_dict[col+"_"+year] = group_potential(potential_dict[year][potential_dict[year].index.get_level_values(1) == "tcs+hoho"], 
                                                      grouping_cols, weight=col+"_potential_pos_overall", mean_cols=mean_cols, 
                                                      sum_cols=sum_cols,  other_cols=other_cols,
                                                      sector="tcs+hoho", drop=[el for el in cols if el != col]).round(2)
        
        cols_potentials = [i+"_"+j for i in [col] for j in pot_cols]

        # Update potential data with availability information from above
        if adjust_potentials:
            ind_dict[col+"_"+year][cols_potentials] = \
                cluster_overall_pot_dict[year][cols_potentials].astype(float).round(2)
            tcs_dict[col+"_"+year][cols_potentials] = \
                cluster_overall_pot_dict[year][cols_potentials].astype(float).round(2)
            hoho_dict[col+"_"+year][cols_potentials] = \
                cluster_overall_pot_dict[year][cols_potentials].astype(float).round(2)
            tcs_hoho_dict[col+"_"+year][cols_potentials] = \
                cluster_overall_pot_dict[year][cols_potentials].astype(float).round(2)
        
        # Add country and bus information (needed in power market model)
        ind_dict[col+"_"+year][col+"_country"] = "DE"
        tcs_dict[col+"_"+year][col+"_country"] = "DE"
        hoho_dict[col+"_"+year][col+"_country"] = "DE"
        tcs_hoho_dict[col+"_"+year][col+"_country"] = "DE"
        
        ind_dict[col+"_"+year][col+"_from"] = "DE_bus_el"
        tcs_dict[col+"_"+year][col+"_from"] = "DE_bus_el"
        hoho_dict[col+"_"+year][col+"_from"] = "DE_bus_el"
        tcs_hoho_dict[col+"_"+year][col+"_from"] = "DE_bus_el"
        
        # Combine to dict holding all parameter data for the clusters
        overall_dict[col+"_"+year] = pd.concat([ind_dict[col+"_"+year], 
                                                tcs_dict[col+"_"+year], 
                                                hoho_dict[col+"_"+year], 
                                                tcs_hoho_dict[col+"_"+year]])
        
        # Adjust rounding: costs to 1 digit; durations to nearest quarter integer; other params to 0 digits
        cols_duration = [i+"_"+j for i in [col] for j in dur_cols]
        cols_costs = [i+"_"+j for i in [col] for j in cost_cols]
        cols_other = [i+"_"+j for i in [col] for j in other_cols]
        
        overall_dict[col+"_"+year][cols_duration] = overall_dict[col+"_"+year][cols_duration].apply(
            round_of_rating)
        overall_dict[col+"_"+year][cols_costs] = overall_dict[col+"_"+year][cols_costs].round(1)
        # Replace zero cost values which might occur in rounding
        overall_dict[col+"_"+year][cols_costs] = overall_dict[col+"_"+year][cols_costs].replace({0.0:0.01})
        overall_dict[col+"_"+year][cols_other] = overall_dict[col+"_"+year][cols_other].round(0)
        
        # Do some renaming
        overall_dict[col+"_"+year].rename(name_dict, inplace=True)
        overall_dict[col+"_"+year].columns = overall_dict[col+"_"+year].columns.str[4:]

In [ ]:
overall_dict["50%_2050"]

In [ ]:
# Info on remaining categories is optionally stored in order to be able to match it with availability time series
# resp. to assign similar availability time series when data is missing.
if write_categories:
    potential_dict["SQ"].to_excel("remaining_categories.xlsx")

In [ ]:
# Write the parameter outputs to Excel
if write_outputs:
    write_multiple_sheets(ind_dict, path_folder_parameterization, filename_out+"_ind.xlsx")
    write_multiple_sheets(tcs_dict, path_folder_parameterization, filename_out+"_tcs.xlsx")
    write_multiple_sheets(hoho_dict, path_folder_parameterization, filename_out+"_hoho.xlsx")
    write_multiple_sheets(tcs_hoho_dict, path_folder_parameterization, filename_out+"_tcs_hoho.xlsx") 
    write_multiple_sheets(overall_dict, path_folder_parameterization, filename_out+"_overall.xlsx") 

## Write outputs
Write outputs needed for the power market model runs separately to csv files

Do some adjustments:
* Simple column naming adjustment
* Timeseries adjustments / harmonization:
> __*NOTE*__: _2017 is used as a simulation year for the power market model._
> * _Availability time series were (mostly) derived for 2012._
> * _Since 2017 and 2012 both started with a Sunday, no shifts of weekdays is necessary._
> * _Only the last day of 2012 is cut here since 2012 was a leap year._
> * _At the end, this only is a very rough estimate / first guess which **should be improved**._

In [ ]:
# Do some column naming adjustment
cases = ["25%_SQ", "50%_SQ", "75%_SQ"]

for case in cases:
    cluster_ts_by_cols[case].columns = cluster_ts_by_cols[case].columns.str[4:]
    ava_pos_ts_by_cols[case].columns = ava_pos_ts_by_cols[case].columns.str[4:-4]
    ava_neg_ts_by_cols[case].columns = ava_neg_ts_by_cols[case].columns.str[4:-4]

    cluster_ts_by_cols[case].rename(name_dict, axis=1, inplace=True)
    ava_pos_ts_by_cols[case].rename(name_dict, axis=1, inplace=True)
    ava_neg_ts_by_cols[case].rename(name_dict, axis=1, inplace=True)

In [ ]:
# Do time series harmonization thing
new_timeindex = pd.date_range(start="2017-01-01 00:00", periods=8784, freq="H")

for case in cases:
    cluster_ts_by_cols[case]["new_timeindex"] = new_timeindex
    ava_pos_ts_by_cols[case]["new_timeindex"] = new_timeindex
    ava_neg_ts_by_cols[case]["new_timeindex"] = new_timeindex
    
    cluster_ts_by_cols[case] = cluster_ts_by_cols[case].set_index("new_timeindex", drop=True).iloc[:8760].round(4)
    ava_pos_ts_by_cols[case] = ava_pos_ts_by_cols[case].set_index("new_timeindex", drop=True).iloc[:8760].round(4)
    ava_neg_ts_by_cols[case] = ava_neg_ts_by_cols[case].set_index("new_timeindex", drop=True).iloc[:8760].round(4)

In [ ]:
cluster_ts_by_cols["25%_SQ"]

In [ ]:
for case in cases:
    # Node information
    overall_dict[case].to_csv(path_folder_parameterization+"sinks_demand_response_el_"+case[:2]+".csv")
    # Load profile information
    cluster_ts_by_cols[case].to_csv(path_folder_parameterization+"sinks_demand_response_el_ts_"+case[:2]+".csv")
    # Availability information
    ava_pos_ts_by_cols[case].to_csv(
        path_folder_parameterization+"sinks_demand_response_el_ava_pos_ts_"+case[:2]+".csv")
    ava_neg_ts_by_cols[case].to_csv(
        path_folder_parameterization+"sinks_demand_response_el_ava_neg_ts_"+case[:2]+".csv")

# Bibliography
Gils, Hans Christian (2015): Balancing of Intermittent Renewable Power Generation by Demand Response and Thermal Energy Storage. Dissertation. Universität Stuttgart, Stuttgart.

Kochems, Johannes (2020): Lastflexibilisierungspotenziale in Deutschland - Bestandsaufnahme und Entwicklungsprojektionen. Langfassung. In: IEE TU Graz (Hg.): EnInnov 2020 - 16. Symposium Energieinnovation. Energy for Future - Wege zur Klimaneutralität. Graz, 12.-14.02, https://www.tugraz.at/fileadmin/user_upload/tugrazExternal/4778f047-2e50-4e9e-b72d-e5af373f95a4/files/lf/Session_E5/553_LF_Kochems.pdf, accessed 11.05.2020.